<a href="https://colab.research.google.com/github/ros-luc/ASFV-epitopes/blob/main/ASFV_Epitopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# **0. Preparation**

---





This notebook is prepared to run in Google Colaboratory, and it will install several packages. 


*In case of running locally, please be advised that many of these steps **are NOT needed** and might **cause problems** in your system.*

**Prerequisites for this pipeline:**

Python packages:

*   Biopython
*   Selenium
*   Pandas
*   Numpy
*   tqdm
*   lxml
*   Scipy
*   scikit-learn 

Additional software:
*   CD-HIT
*   MUSCLE
*   IEDB MHC I, MHC II and B-cell prediction tools
*   NetChop
*   NetSurfP

### Declaring base variables

In [ ]:
#These variables are specific to the organism we are studying

#UniProt proteomes query
UNIPROT_QUERY = "ASFV"

#In case we want a particular proteome to be present in all clusters, we can
#specify it here. Otherwise, leave it as an empty string.
mustHaveProteome = "UP000141072"

#We need the ID of our organism at IEDB. For example, for ASFV is 10497; for
#T. cruzi is 5693. It can be found at IEDB.org and searching in "Organism".
IEDB_ID = 10497

### Making base directories

In [ ]:
!mkdir -p ./RV/{DATA/{RAW,PROCESSED/{"T_PREDICTIONS","B_PREDICTIONS"}},TOOLS}
%cd /content/RV

### Installing Miniconda

We are installing **version 4.5.4**, since it runs on **Python 3.6**.

**All other packages that we install have to be for Python 3.6 in order to avoid conflicts.**

In [ ]:
!which python3
!python --version

In [ ]:
%%shell
wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
chmod +x Miniconda3-4.5.4-Linux-x86_64.sh #We have to make the installer executable
bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
rm "/content/RV/Miniconda3-4.5.4-Linux-x86_64.sh"

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

In [ ]:
!which python3
!python --version

### Installing Biopython

Biopython includes many useful tools to parse biological sequences and results from tools such as BLAST or MSAs

In [ ]:
!conda install -c conda-forge biopython python=3.6.5 -y -q 

### Installing and configuring Selenium

In [ ]:
!conda install selenium python=3.6.5 -y -q

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

### Installing CD-HIT

In [ ]:
!conda install -c bioconda cd-hit python=3.6.5 -y -q 

### Installing MUSCLE

In [ ]:
%%shell
wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz
tar -xzvf muscle3.8.31_i86linux64.tar.gz
cp muscle3.8.31_i86linux64 /usr/local/bin #We copy it here so that it can run properly
chmod 755 /usr/local/bin/muscle3.8.31_i86linux64 #We change its permissions too.

rm "/content/RV/muscle3.8.31_i86linux64"
rm "/content/RV/muscle3.8.31_i86linux64.tar.gz"

### Additional packages

Some of these are needed for immunoinformatics tools.

In [ ]:
!sudo apt-get -qq install tcsh
!sudo apt-get -qq install gawk
!sudo apt-get -qq install gfortran
!sudo apt-get -qq install csh

### Importing Python packages

In [ ]:
#General use
import os #To work with files and directories
import shutil #To move files and directories
import requests #To work with html data
from lxml import html #To work with html data
import time #Time functions
from time import sleep #Sleep function for delays
from tqdm.notebook import tqdm #Progress bar
import subprocess #Command line
import csv #To read csv files
from multiprocessing.pool import ThreadPool #To run multiprocessing functions
from selenium.webdriver.common.action_chains import ActionChains #Needed for Selenium
from openpyxl import load_workbook #Work with .xlsx files
from io import StringIO
from collections import Counter
from collections import defaultdict


#Biopython
from Bio import AlignIO
from Bio import SeqIO
from Bio import pairwise2
from Bio.Align.Applications import MuscleCommandline
from Bio.Align.AlignInfo import SummaryInfo
from Bio.Align import AlignInfo
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

#Pandas and similar
import pandas as pd 
import scipy as sc
import scipy.stats
import numpy as np

#Google drive files
from google.colab import files

In [ ]:
#We will also use this function several times

def createPath(pathToCreate):
    if not os.path.exists(pathToCreate):
        os.makedirs(pathToCreate)
    return str(pathToCreate)



---


# **1. Downloading protein sequences**



---







We will download all protein entries from the proteomes of our target organism. This section looks for proteomes in UniProt and downloads all the corresponding protein sequences.

This section is skippable if you already have a "master" fasta file with all proteomes and proteins.

## **1a. Downloading all Uniprot proteomes**

### **List of proteomes**

First, we search for all Uniprot proteomes matching our query and download them. Be sure to test this query beforehand to avoid downloading unwanted proteomes.

We also want to fill our proteome list with additional features, such as host, collection date, etc...


In [ ]:
def DownloadProteomes_Uniprot(query):

    #First, we will download the list of proteomes and some information about them
    print(f"Getting list of all {query} proteomes...", end="")
    base_url = "http://www.uniprot.org/proteomes/"
    payload = {"query" : query, "format" : "tab", "columns" : "id,name,organism-id,mnemonic,proteincount"}
    raw_result = requests.get(base_url, params=payload)

    if raw_result.ok:
        result = StringIO(raw_result.text)
        df = pd.read_csv(result, sep="\t")
    else:
        print("Something went wrong: ", result_proteome.status_code)
    print("done!")
    
    #We want the accession number of these proteomes to get some more info
    print("Getting accession numbers...", end="")
    accessionDict = {}
    for proteome in tqdm(df["Proteome ID"]):
        url = f'https://www.uniprot.org/proteomes/{proteome}'
        path = '//*[@id="results"]/tbody/tr/td[3]/div/div[1]/a'
        response = requests.get(url)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path)
        if not tree:
            print(f"Accession for {proteome} not available")
        else:
            accessionDict[str(tree[0].text_content())] = proteome
    df["Accession number"] = accessionDict

    #In these dictionaries we will store the new data
    countryDict = {}
    hostDict = {}
    strainDict = {}
    dateDict = {}
    seroDict = {}
    isolateDict = {}
    isolatezoneDict = {}
    
    listofdicts = [countryDict,
                hostDict,
                strainDict,
                dateDict,
                seroDict,
                isolateDict,
                isolatezoneDict
                ]

    print("Getting proteome features...", end="")
    for access in tqdm(df["Accession number"]):
        
        #We give an empty default value to each dictionary
        for dictionary in listofdicts:
            dictionary[access] = ""
        
        url = f"https://www.ebi.ac.uk/ena/browser/api/embl/{access}"
        response = requests.get(url)
        
        if response.ok:
            for line in response.iter_lines():
                
                #Here is a custom parser for getting additional information
                if "host=" in str(line):
                    hostDict[access] = str(line).split("host=")[1].replace('"',"").replace("'","")
                    continue  
                
                if "/lab_host=" in str(line):
                    hostDict[access] = str(line).split("/lab_host=")[1].replace('"',"").replace("'","")
                    continue

                if "/country=" in str(line):
                    countryDict[access] = str(line).split("/country=")[1].replace('"',"").replace("'","")
                    continue
                
                if "/strain=" in str(line):
                    strainDict[access] = str(line).split("/strain=")[1].replace('"',"").replace("'","")
                    continue
                
                if "/collection_date=" in str(line):
                    dateDict[access] = str(line).split("/collection_date=")[1].replace('"',"").replace("'","")
                    continue

                if "/serotype=" in str(line):
                    seroDict[access] = str(line).split("/serotype=")[1].replace('"',"").replace("'","")
                    continue

                if "/isolate=" in str(line):
                    isolateDict[access] = str(line).split("/isolate=")[1].replace('"',"").replace("'","")
                    continue

                if "/isolation_source=" in str(line):
                    isolatezoneDict[access] = str(line).split("/isolation_source=")[1].replace('"',"").replace("'","")
                    continue

    #Finally, we map this new information in the dataframe
    df['host'] = df['Accession number'].map(hostDict)
    df['date'] = df['Accession number'].map(dateDict)
    df['country'] = df['Accession number'].map(countryDict)
    df['strain'] = df['Accession number'].map(strainDict)
    df['isolate'] = df['Accession number'].map(isolateDict)
    df['isolation source'] = df['Accession number'].map(isolatezoneDict)
    df['serotype'] = df['Accession number'].map(seroDict)
  
    print(f"Downloaded {str(len(df))} {query} proteome entries.")

    return df

In [ ]:
try:
    df_proteomes = DownloadProteomes_Uniprot(UNIPROT_QUERY)
    df_proteomes.to_excel("./DATA/RAW/Proteomes.xlsx")
    df_proteomes
except:
    raise Exception("Unable to download proteomes.")

### **Assessing proteomes**

#### Problematic proteomes

Some proteomes give aberrant alignments due to being too fragmented or incomplete. This gives problems down the line during the entropy calculations, so we will discard them.

In [ ]:
#These are specific for African Swine Fever Virus. Modify list elements as needed.

problematic_proteomes = ["UP000321214", #Kyev 2016 (245 proteins)
                         "UP000274966", #Poland 2016 o23 (17 proteins)
                         "UP000268777", #Poland 2016 o9 (11 proteins)
                         "UP000282187", #Poland 2017 C220 (5 proteins)
                         "UP000278405", #Poland 2016 o10  (3 proteins)
                         "UP000266411", #Sardinia Sassari 2008 (234 proteins)
                         "UP000423628", #South Africa 1985/SPEC 57 (74 proteins)
                        ]

for proteome in problematic_proteomes:
    df_proteomes = df_proteomes.drop(df_proteomes.loc[df_proteomes['Proteome ID']==proteome].index)

## **1b. Downloading protein sequences**

We download all the proteins from each proteome, and tag them with their proteome origin. We then append each protein to a new fasta file, which will contain all the proteins from all proteomes.

The resulting fasta entries will have the format `>PROTEOME-ID_PROTEIN-ID`


In [ ]:
def fetch_proteome(proteome):
    base_url = "http://www.uniprot.org/uniprot/"
    payload = {"query": "proteome:" + proteome,
               "format": "fasta",
               "include": "yes"
                }

    result = requests.get(base_url, params = payload)
    if result.ok:
        data = {proteome : result.text}
        print("Downloaded " 
              + result.headers["X-Total-Results"] 
              + " proteins for proteome " 
              + proteome
              )
        return data
    else:
        print("Could not download proteome " + proteome)
        return None

In [ ]:
allproteomes = list(df_proteomes["Proteome ID"])

for i in range(1,10):
    try:
        results = ThreadPool(len(allproteomes)).imap_unordered(fetch_proteome, allproteomes)
        for proteome in results:
            for key, value in proteome.items():
                prots = value.split(">")[1:]
                for entry in prots:
                    uniprotID = entry.split("\n")[0].split("|")[1]
                    sequence = "".join(entry.split("\n")[1:])
                    with open("./DATA/RAW/allproteins.fa", "a+") as f:
                        f.write(">" + key + "_" + uniprotID)
                        f.write("\n")
                        f.write(sequence)
                        f.write("\n")
        break
    except:
        print("Connection error")
        time.sleep(30)
                 
print("All proteins downloaded!")

---
# **2. Prediction of T-cell epitopes**

---

## **2a. Protein clustering**

### **cd-hit**

We run the program *CD-HIT* to create protein clusters.

```
-i --> Input file
-o --> Output
-d 0 --> Full description of cluster name until first space
-c --> % identity cutoff threshold (set to 0.8, deafult is 0.9)
-s --> % of length for smaller sequences, we set to 0.75 to avoid strange alignments of short proteins (default 0)
-n --> word size
```





The input file must be a FASTA file with all the protein sequences to cluster.

For this pipeline, each entry has to be in the format `>PROTEOME-ID_PROTEIN-ID`.

e.g.:

```
>UP000000861_P0C9M9
MNSLQVLTKKVLIENKAFSNYHEDDSFILQQLGLWWENGPIGFCKQCKMVTGGSMLCSDVDSYELDRALVKAVKENQTDL...
>UP000000861_P0C9Q0
MLPSLQSLTKKVLAGQCLPEDQHYLLKCYDLWWNNAPITFDHNLRLIKSAGLQEGLDLNMALVKAVKENNYSLIKLFTEW...
>UP000000861_P0C9K9
MITLYEAAIKTLITHRKQILKHPDSREILLALGLYWNKTHILLKCHECGKISLTGKHSTKCININCLLILAIKKKNKRMV...
>UP000000861_P0C8F5
MKMHIARDSIVYLLNKHLQNTILTNKIEQECFLQADTPKKYLQYIKPFLINCMTKNITTDLVMKDSKRLEPYITLEMRDI...
```



In [ ]:
#We specify an identity cutoff at 80%, and a minimum length for smaller sequences of 75%

!cd-hit -i ./DATA/RAW/allproteins.fa -o "./DATA/PROCESSED/T_PREDICTIONS/globalclusters" -d 0 -c 0.8 -s 0.75

### **Cluster filtering**

We will filter our clusters according to:

1.   **Number of proteomes in each cluster**

    We want a minimum representation of all strains in our clusters.


2.   **Presence of a particular proteome in each cluster**

    In case we want a particular proteome to be present always.


In [ ]:
def parseClusters(clstrfile):
    returndict = {}
    print(f"Parsing clusters for file {os.path.basename(clstrfile)}:")

    with open(clstrfile, "r") as f:
        for line in f:
            if line.startswith(">"):
                clusterID = line.strip()
                returndict[clusterID] = []
            else:
                returndict[clusterID].append(line.strip())
    
    print(f"Total number of clusters: {str(len(returndict))}")

    return returndict

def filterClusters(clstrfile,
                   min_proteomes, #Minimum number of proteomes in each cluster
                   mustProteome=""
                   ):
    
    clusterDict = parseClusters(clstrfile)
  
    for key, value in list(clusterDict.items()): #we have to make it a list so we can delete keys
        proteomes = []
        for i in value:
            proteomes.append(i.split()[2].split("_")[0][1:])

        if len(proteomes) < min_proteomes:
            try:
                del clusterDict[key]
            except:
                pass

        if mustHaveProteome:
            if not mustHaveProteome in proteomes:
                try:
                    del clusterDict[key]
                except:
                    pass

    print("Filtered number of clusters: " + str(len(clusterDict)))

    return clusterDict

In [ ]:
global_clusters = filterClusters("./DATA/PROCESSED/T_PREDICTIONS/globalclusters.clstr",
                                 min_proteomes=14,
                                 mustProteome=mustHaveProteome
                                 )

### **Reconstructing FASTAS**


Now we build new FASTA files from these clusters.

We are also going to define a list with proteomes to give priority to in the MSAs, and add a prefix to each protein accordingly.

In [ ]:
#In this case, we will give priority to proteomes from genotype II, the current
#responsible of ASFV epidemic. We will give two tiers of importance: first 
#Georgia, and then the rest of genotype II.

priorityProteomesList = ["UP000141072", #Georgia 2007/1
                    "UP000345145", #Georgia 2008/2
                    "UP000267045", #Estonia 2014
                    "UP000326051", #Lithuania 2014 
                    "UP000325567", #Moldova 2017
                    "UP000327056", #CzechRepublic 2017/1
                    "UP000290386", #China/2018/AnhuiXCGQ
                    "UP000291821", #China Pig/HLJ/2018
                    "UP000292678", #China DB/LN/2018
                    "UP000307568", #Belgium 2018/1
                    "UP000316600", #China ASFV-wbBS01
                    "UP000324915", #Belgium Etalle 2018
                    "UP000428265", #Hungary ASFV_HU_2018
                    "UP000422299", #China/CAS19-01/2019 
                    ]

priorityProteomesDict = {}
for proteome in priorityProteomesList:
    if proteome == mustHaveProteome:
        priorityProteomesDict[proteome] = "01"
    else:
        priorityProteomesDict[proteome] = "02"

In [ ]:
#We will parse the original file with all proteins in order to create a complete
#dictionary that contains all sequences in a format "proteinID : sequence"

with open("./DATA/RAW/allproteins.fa", "r") as f:
    allProteins = {record.id:str(record.seq) for record in SeqIO.parse(f, "fasta")}

In [ ]:
def reconstruct_fasta(clusterDictionary, savePath, proteomePriorityDict):
    clusterName = os.path.split(os.path.dirname(savePath))[-1]

    for cluster, members in clusterDictionary.items():
        clusterProteinList = []
        for member in members:
            clusterProteinList.append(member)
        
        fileName = f"{clusterName}_{cluster[1:]}.fa"
        filePath = os.path.join(savePath, fileName) 

        with open(filePath, "w+") as f:
            for protein in clusterProteinList:
                fullProt = protein[protein.find(">") + 1 : protein.find("...")]
                protProteome = fullProt.split("_")[0]
                protID = fullProt.split("_")[1]

                #Here we asign each proteome a priority by adding a prefix
                if protProteome in proteomePriorityDict:
                    f.write(f">{proteomePriorityDict[protProteome]}_{protProteome}_{protID}\n")
                else:
                    f.write(f">99_{protProteome}_{protID}\n")

                f.write(allProteins[fullProt])
                f.write("\n")

    print (f"FASTAS for {clusterName} reconstructed.")

In [ ]:
globalPath = createPath("./DATA/PROCESSED/T_PREDICTIONS/FASTAS_CLUSTERS/GLOBAL/")
reconstruct_fasta(global_clusters, globalPath, priorityProteomesDict)

## **2b. Multiple sequence alignment**

### **Running MUSCLE**

We run the MUSCLE MSA tool for each cluster. It results in a new MSA fasta file stored in MUSCLE ALIGNMENTS.

In [ ]:
def runMUSCLE(infile, outfile):
    #Here is where we installed MUSCLE in section 0. Modify as needed.
    muscle_exe = r"/usr/local/bin/muscle3.8.31_i86linux64"

    muscle_cline = MuscleCommandline(muscle_exe,
                                     input=infile,
                                     out=outfile,
                                     clwstrict=True
                                    )
    muscle_cline()

def alignDir(fastas_dir):
    dirname = os.path.basename(os.path.normpath(fastas_dir))

    print(f"Aligning {dirname} clusters")
    print("-"*len(f"Aligning {dirname} clusters"))

    out_path = createPath(f"./DATA/PROCESSED/T_PREDICTIONS/MUSCLE_ALIGNMENTS/UNSORTED/{dirname}")

    for root, dirs, files in os.walk(fastas_dir):
        for fasta in tqdm(files):
            if fasta.endswith(".fa"):
                in_file = os.path.join(root, fasta)
                out_file = out_path + "/MSA_" + fasta
                runMUSCLE(in_file, out_file)
    print("-"*len(f"All MSAs for {str(dirname)} done!"))    
    print(f"All MSAs for {str(dirname)} done!")
    print("-"*len(f"All MSAs for {str(dirname)} done!"))

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/T_PREDICTIONS/FASTAS_CLUSTERS/"):
    for dir in dirs:
        if not dir.startswith("."):
            alignDir(os.path.join(root, dir))

### **Sorting the MSAs**

We will now open each MSA and sort it first according to our proteome priority, and then by number of positions aligned.

In [ ]:
def keyMSA(item):
    #We need a custom function to sort the MSA by priority and length of alignment
    returnList = []
    returnList.append(1 - int(item.id.split("_")[0])) #(1 - x) because descending order
    returnList.append(len(item) - item.seq.count("-"))
    return tuple(returnList)

def sortMSA(msafile):
    alignment = AlignIO.read(msafile,"clustal")
    alignment.sort(key = keyMSA, reverse=True)
    return alignment

def sortMSA_dir(in_folder):
    musclefiles = [os.path.join(in_folder, x) for x in os.listdir(in_folder) if x.endswith(".fa")]
    clusterName = os.path.basename(os.path.normpath(in_folder))
    dir_sorted = createPath(f"./DATA/PROCESSED/T_PREDICTIONS/MUSCLE_ALIGNMENTS/SORTED/{clusterName}/")

    for m in musclefiles:
        aln = sortMSA(m)
        out_aln_file = os.path.basename(m).replace(".fa", "_sorted.aln")
        out_aln_path = os.path.join(dir_sorted, out_aln_file)
        with open(out_aln_path,"w+") as outfile:
            AlignIO.write(aln, outfile, "clustal")

    print(f"Sorted {str(len(musclefiles))} {clusterName} alignments")

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/T_PREDICTIONS/MUSCLE_ALIGNMENTS/UNSORTED"):
    for dir in dirs:
        if not dir.startswith("."):
            msa_dir = os.path.join(root,dir)
            sortMSA_dir(msa_dir)

## **2c. Entropy and consensus sequence**

### **Calculate entropy and generate consensus sequences**

We will use the Shannon entropy to calculate the variability in each position of each MSA. If a position has an entropy above the selected threshold (by default 0), it will be masked with an asterisk(*), thus generating a masked consensus sequence.

Then, we will find the sequence from the MSA that most closely resembles the masked consensus, and mask the same positions as the masked consensus. This way, we end up with a "real" masked sequence, not a consensus "artificial" one. However, if the selected threshold is 0, all the sequences will resemble the masked consensus equally, in which case priotitized proteomes take preference.

In [ ]:
def shannonEntropy(data):
    data_series = pd.Series(data)
    counts = data_series.value_counts()
    shannon_entropy = scipy.stats.entropy(counts)
    return shannon_entropy

def Shannon(alnfile, shannon_threshold=0):   
    clusterName = os.path.dirname(alnfile).split("/")[-1]
    shannon_str = str(shannon_threshold).replace(".", "")
    dest_pref = f"./DATA/PROCESSED/T_PREDICTIONS/CONSENSUS/SHANNON_{shannon_str}"
    dest_dir = createPath(f"{dest_pref}/{clusterName}/")
    dest_file = os.path.basename(alnfile).replace("sorted.aln",
                                                  f"consensus{shannon_str}.fasta"
                                                )
    
    #It will go position by position, calculate the entropy and then create a
    #consensus sequence with the most common amino acid, unless the entropy is 
    #higher than the threshold, in which case it will insert an *.
    consensus_list = []
    alignment = AlignIO.read(alnfile, "clustal")
    for i in range(0,alignment.get_alignment_length()):
        aa = []
        for record in alignment:
            aa.append(record.seq[i])
        position_entropy = shannonEntropy(aa)
        if position_entropy <= shannon_threshold:
            consensus_list.append(Counter(aa).most_common()[0][0])
        else:
            consensus_list.append("*")
    consensus_seq = "".join(consensus_list)

    #To find the closest sequence to the consensus, we will do a pairwise alignment
    #between the consensus and all sequences from the MSA. The sequence with highest
    #score will be the closest one. In case of equal score, prioritized proteomes
    #have advantage.
    refSeq = None
    highestScore = 0
    for record in alignment:
        pairwise_score = pairwise2.align.globalxx(record.seq,
                                                consensus_seq,
                                                score_only=True
                                                )
        if pairwise_score > highestScore:
            highestScore = pairwise_score
            refSeq = record
        elif pairwise_score == highestScore:
            if int(record.id[0:2]) < int(refSeq.id[0:2]):
                refSeq = record
    
    #Finally, we will mask the positions of the closest sequence according to
    #the masked consensus sequence.
    refSeq_aa = list(refSeq.seq)
    masked_ref = ["*"] * len(refSeq_aa)
    for i in range(0, len(refSeq_aa)):
        if consensus_seq[i] != "*":
            masked_ref[i] = refSeq_aa[i]
    
    #We write the results in a new fasta file
    with open(dest_dir + dest_file, "w+") as f:
        f.write(f">{refSeq.id}\n")
        f.write("".join(masked_ref))

    #All results from the same cluster type and Shannon threshold will be appended
    #in a file.
    mergedFile = (f"{dest_pref}/{clusterName}_fullconsensus{shannon_str}.fasta")
    with open(mergedFile, "a+") as f:
        f.write(f">{refSeq.id}\n")
        f.write("".join(masked_ref)+"\n")

In [ ]:
#In case we want to calculate the consensus for different thresholds, we can
#put them in a list here:
shannon_thresholds = [0,
                      #0.25,
                      0.5
                      ]

for s in shannon_thresholds:
    for dir in os.listdir("./DATA/PROCESSED/T_PREDICTIONS/MUSCLE_ALIGNMENTS/SORTED"):
        if not dir.startswith("."):
            cluster_dir = os.path.join("./DATA/PROCESSED/T_PREDICTIONS/MUSCLE_ALIGNMENTS/SORTED", dir)
            print(f"Calculating invariable proteome at Shannon {s} for {dir}")
            for f in tqdm(os.listdir(cluster_dir)):
                if f.endswith(".aln"):
                    Shannon(os.path.join(cluster_dir, f), s)

## **2d. Validation of IEDB T-cell epitopes**

### **Downloading IEDB epitopes**

We will download the available confirmed epitopes of our organism from IEDB via Selenium.

In [ ]:
%cd -q "./DATA/RAW"
try:
    wd = webdriver.Chrome('chromedriver', options=options)
    wd.get(f"http://www.iedb.org/sourceOrgId/{IEDB_ID}")

    wd.find_element_by_xpath('//*[@id="content"]/div/div[2]/table/tbody/tr[5]/td[2]/div/a').click() #Click on T assays link
    time.sleep(5)

    wd.find_element_by_css_selector('#refine_container > div:nth-child(8) > div.search_content > div.ui_input.check.search_bothlinks > div.checkbox.square').click() #Positive assays only
    time.sleep(5)

    wd.find_element_by_css_selector('#refine_container > button:nth-child(4)').click() #Search
    time.sleep(5)

    wd.find_element_by_css_selector('#result_carousel > div.carousel_content.active > div > div.contenttable_content.active > div > div:nth-child(1) > div.exportholder').click() #Export results
    time.sleep(5)

    wd.find_element_by_css_selector("div.inwindow_popup.drop_arrow.identifier_container.show > div > div:nth-child(1) > div.exportholder > div.txt").click() #Popup Export to CSV file
    time.sleep(5)

    wd.switch_to.window(wd.window_handles[-1]) #Switch to "download" page
    time.sleep(30)

    wd.refresh() #We refresh to prompt the download
    time.sleep(2)

    wd.close()

    %cd -q ../../

    for f in os.listdir("./DATA/RAW/"):
        if f.endswith(".zip"):
            if f.startswith("tcell"):
                zipname = "./DATA/RAW/" + f
                !unzip -qq $zipname -d "./DATA/RAW/"
                os.remove(zipname)

    for f in os.listdir("./DATA/RAW/"):
        if f.endswith(".csv"):
            if f.startswith("tcell"):
                os.rename(r"./DATA/RAW/" + f, r"./DATA/RAW/T_Cell_IEDB_epitopes.csv")

    IEDB_T_epitopes = {}
    with open("./DATA/RAW/T_Cell_IEDB_epitopes.csv","r") as f:
        reader = csv.reader(f)
        next(reader)
        next(reader)
        for line in reader:
            IEDB_T_epitopes[line[0]] = (line[11], #Sequence
                                        line[15], #Antigen
                                        line[17], #Antigen ID
                                        line[101], #MHC allele
                                        line[102], #MHC class
                                        line[84], #Assay 1
                                        line[85], #Assay 2
                                        line[87]  #Assay 3
                                        )
    
    print("IEDB T-cell epitopes succesfully downloaded.")
except:
    %cd -q ../../
    print("IEDB epitopes could not be downloaded.")

### **Identify conserved IEDB epitopes**

In [ ]:
def IEDB_T_conserved(consensus_file, epitopes_dict):
    records = list(SeqIO.parse(consensus_file, "fasta"))

    conservedEpis = []

    for record in records:
        sequence = str(record.seq)
        for epitope_id, epitope_data in epitopes_dict.items():
            if epitope_data[0] in sequence:
                epitope_entry = [epitope_data[0],
                                 record.id,
                                 epitope_data[1],
                                 epitope_data[2],
                                 f"Class {epitope_data[4]} ({epitope_data[3]})",
                                 " ".join(epitope_data[3:])]
                if epitope_entry[4] == "Class  ()":
                    epitope_entry[4] = "NA"
                conservedEpis.append(tuple(epitope_entry))
    
    df = pd.DataFrame(conservedEpis,
                      columns=['Epitope',
                               'Antigen',
                               'IEDB_Antigen name',
                               'IEDB_Antigen ID',
                               'MHC allele',
                               'IEDB_Assay'
                               ]
                      )

    clusterN = os.path.basename(consensus_file).split("_")[0]
    shannonN = os.path.dirname(consensus_file).split("/")[-1].split("_")[-1]

    output_path = createPath(consensus_file.split("CONSENSUS")[0] + "IEDB/")

    output_path = "".join(output_path 
                          + clusterN
                          + "_"
                          + shannonN
                          + "_IEDB_epis.csv"
                          )

    df.to_csv(output_path)

In [ ]:
consensusFolder = "./DATA/PROCESSED/T_PREDICTIONS/CONSENSUS/"

for shannon in os.listdir(consensusFolder):
    for f in os.listdir(consensusFolder + shannon):
        if f.endswith(".fasta"):
            IEDB_T_conserved(os.path.join(consensusFolder + shannon, f), IEDB_T_epitopes)

## **2e. *De novo* epitope predictions**

### **CD8 - MHC I epitopes**

#### **Proteasome predictions**


NetChop simulates how proteins are digested in the proteasome, so we will run it against our conserved sequences.

First, you need to download the official NetChop Linux .tar.gz package from DTU.

https://services.healthtech.dtu.dk/

This pipeline was tested using NetChop version 3.1d for Linux. Other versions might need changes in the code.

In [ ]:
#Upload the .tar.gz file, uncompress and untar it.

!mkdir -p ./TOOLS/NETCHOP/
from google.colab import files
netchop_pack = files.upload()
netchop_filename = next(iter(netchop_pack))
!tar -xzf {netchop_filename} -C ./TOOLS/NETCHOP
!rm {netchop_filename}

for dir in os.listdir("./TOOLS/NETCHOP/"):
    if not dir.startswith("."):
        NetChopDir = f"./TOOLS/NETCHOP/{dir}"
        fullNetChopPath = f"./TOOLS/NETCHOP/{dir}/netchop"

In [ ]:
#Modify the netchop script settings following the README file instructions

parsedNetChop = []
with open(fullNetChopPath, "r") as f:
    for line in f:
        parsedNetChop.append(line.strip("\n"))

for counter, line in enumerate(parsedNetChop):
    if line.startswith("setenv  NMHOME"):
        parsedNetChop[counter] = f"setenv  NMHOME  {os.path.abspath(NetChopDir)}"
    if line.startswith("        setenv  TMPDIR"):
        parsedNetChop[counter] = f"        setenv  TMPDIR  {os.path.abspath(NetChopDir)}/tmp"

with open(fullNetChopPath, "w+") as f:
    for line in parsedNetChop:
        f.write(line + "\n")

!chmod 755 {fullNetChopPath}

Now that we have it installed, we can proceed with proteasomal predictions.

In [ ]:
def NetChop(in_file):
    
    shutil.copy(in_file, "NetChopTemp.fasta")
    
    shannon = in_file.split("/")[-3]
    cluster = in_file.split("/")[-2]

    protein_entry = SeqIO.read(in_file, "fasta")
    seqID = protein_entry.id
    maskSeq = str(protein_entry.seq)

    result = subprocess.run([fullNetChopPath, '-t 0.5', '-v 0', 'NetChopTemp.fasta'], stdout=subprocess.PIPE)
    netchopOutput=result.stdout.decode().split("\n")
    with open("NetChopOutput.txt", "w+") as f:
        f.write(result.stdout.decode())
    iter_netchopOutput = iter(netchopOutput)

    chopValues = {}
    time.sleep(0.5)
    for line in iter_netchopOutput:
        if "pos" in line:
            line = next(iter_netchopOutput)
            line = next(iter_netchopOutput)
            while "----" not in line:
                chopValues[int(line.split()[0])] = line.split()[2]
                line = next(iter_netchopOutput)

    parsedList = []
    for position, aa in enumerate(maskSeq, 1):
        if aa == "*" or aa == "-" or aa == "X":
            parsedList.append(aa)
        else:
            if chopValues[position] == "S":
                parsedList.append(aa + "_")
            else:
                parsedList.append(aa)
    parsedSeq = "".join(parsedList)

    savePath = createPath("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8/NETCHOP/" 
                          + shannon
                          + "/"
                          )
    saveFile = f"{cluster}_chopconsensus{shannon.split('_')[-1]}.fasta"

    with open(savePath + saveFile, "a+") as outputfile:
        outputfile.write(f">{seqID}\n")
        outputfile.write(parsedSeq + "\n")
  
    os.remove("NetChopTemp.fasta")
    os.remove("NetChopOutput.txt")

In [ ]:
def LaunchNetChop(filelist):
    #We need a list of all the files to be processed. They need to be individual
    #fasta files

    print("Launching NetChop...")
    print("")

    try:
        for x in tqdm(filelist):
            NetChop(x)
    except:
        print("Something went wrong, retrying...")
        !rm -rf "./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8/NETCHOP/"
        for x in tqdm(filelist):
            NetChop(x)

    print("Finished!")

In [ ]:
#We will launch NetChop against the individual masked files
consPath = "./DATA/PROCESSED/T_PREDICTIONS/CONSENSUS/"
maskedConsensusFiles = []
for root, dirs, files in os.walk(consPath):
    for f in files:
        if f.startswith("MSA"):
            maskedConsensusFiles.append(os.path.join(root, f))

LaunchNetChop(maskedConsensusFiles)

#### **Generate peptides of 9 aminoacids**

MHCI presenting peptides usually have a lenght of 9 aminoacids. We will separate the proteasomal results into conserved peptides with a length equal or bigger than 9 aminoacids. Then, we will create all possible 9-mer overlapping epitopes for those sequences larger than 9 amino acids.

In [ ]:
def Peptidator9(in_folder):

    shannon = in_folder.split("_")[-1]
    choppedFileList = [os.path.join(in_folder, f) for f in os.listdir(in_folder) if f.endswith(".fasta")]

    for f in choppedFileList:
        cluster = f.split("/")[-1].split("_")[0]
        consensusDict = {}
        with open(f,"r") as consensus:
            for line in consensus:
                if line.startswith(">"):
                    id = line.strip()
                    consensusDict[id] = ""
                else:
                    consensusDict[id] += line.strip()
                    
        for key, value in consensusDict.items():
            fragments = value.replace("_", " ").replace("*", " ").replace("-", " ").split()
            consensusDict[key] = [peptide for peptide in fragments if len(peptide) >= 9]

        output_filename = f"{cluster}_{shannon}_peptides9.fasta"

        with open(os.path.join(in_folder, output_filename), "w+") as output:
            for key, values in sorted(consensusDict.items()):
                for counter, value in enumerate(values, 1):
                    output.write(f"{key}_{str(counter)}\n")
                    output.write(value + "\n")

In [ ]:
def Nonamerizator(fastafile):
    peptides = list(SeqIO.parse(fastafile,"fasta"))
    fastabasename = os.path.basename(fastafile).replace(".fasta", "_nonamers.fasta")
    output = createPath("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8/") + fastabasename
    open(output,"w+").close()
    
    for record in peptides:
        ninemers = []
        for position, aa in enumerate(record.seq):
            ninemers.append(record.seq[position:position+9])
            ninemers = [x for x in ninemers if len(x)==9]
        
        for counter, n in enumerate(ninemers,1):
            with open(output,"a+") as f:
                f.write(f">{str(record.id)}_{str(counter)}\n")
                f.write(str(n) + "\n")

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8/NETCHOP/"):
    for dir in dirs:
        Peptidator9(os.path.join(root, dir))
        for f in os.listdir(os.path.join(root, dir)):
            if f.endswith("peptides9.fasta"):
                Nonamerizator(os.path.join(root, dir, f))

#### **TAP predictions**

We will use the server by Louzoun to run the TAP predictions, using Selenium.

In [ ]:
def prepareTAPfile(peptide_dir, dirTAP):
    #Epitope predictions are independent of their origin, so we merge all
    #9mers into the same file.
    output_file = os.path.join(dirTAP, "TAP_peptides.txt")
    joinedPeptideList = []
    fastas_list = [os.path.join(peptide_dir, x) for x in os.listdir(peptide_dir) if x.endswith(".fasta")]
    for fasta_file in fastas_list:
        peptideList = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
        for x in peptideList:
            joinedPeptideList.append(x)
    uniquePeptideList = set(joinedPeptideList)

    with open(output_file, "w+") as f:
        for peptide in uniquePeptideList:
            f.write(peptide + "\n")
    
    return output_file

def serverTAP(dirTAP, peptideFile):
    currentPath = os.path.abspath(os.getcwd())
    peptideFileABS = os.path.abspath(peptideFile)
    resultsPath = createPath(os.path.join(dirTAP, "RESULTS"))
    
    %cd -q $resultsPath

    wd = webdriver.Chrome('chromedriver', options=options)
    
    print("Connecting to TAP binding app...", end="")
    wd.get("https://transition-probability.shinyapps.io/TAP_binding_App/")
    time.sleep(20)
    print("ok.")
    
    print("Uploading peptides...", end="")
    wd.find_element_by_id("file").send_keys(peptideFileABS)
    time.sleep(20)
    print("ok.")
    
    print("Downloading predictions...", end="")
    wd.find_element_by_id("downloadData").click()
    time.sleep(20)
    print("ok.")
    wd.quit()
    print("Predictions done!")

    %cd -q $currentPath

    for f in os.listdir(resultsPath):
        if f.endswith(".csv"):
            TAPresults = os.path.join(resultsPath, f)
            return TAPresults
    return False

def parseTAP(tapcsv, peptideFile):
    print("Parsing file...", end="")  
    TAPdict = {}
    
    nonamersList = []
    with open(peptideFile, "r") as pepf:
        for line in pepf:
            nonamersList.append(line.strip())

    with open(tapcsv, "r") as tapf:
        reader = csv.reader(tapf, delimiter=',', quotechar='"')
        next(reader)
        for counter, line in enumerate(reader):
            if line == "":
                break
            else:
                TAP_prob = float(line[1].replace(",","."))
                TAP_IC50 = float(line[2].replace(",","."))
                TAPdict[nonamersList[counter]] = (TAP_prob, TAP_IC50)
    print("done!")
    
    return TAPdict

def TAP(peptides_folder):
    TAP_folder = createPath(os.path.join(peptides_folder, "TAP/"))
    
    TAP_peptides = prepareTAPfile(peptides_folder, TAP_folder)

    results = serverTAP(TAP_folder, TAP_peptides)

    if not results:
        raise Exception("CSV not found.")
    else:
        TAP_dict = parseTAP(results, TAP_peptides)
    
    return TAP_dict

In [ ]:
TAPdict = TAP("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8/")

#### **Predicting peptide affinity for MHC I**

We first download the MHC I binding tool from the IEDB.

In [ ]:
mhci_url = "https://downloads.iedb.org/tools/mhci/3.1/IEDB_MHC_I-3.1.tar.gz"
mhci_filename = mhci_url.split("/")[-1]

print("Downloading IEDB MHC I tool...")
!wget $mhci_url
!mkdir -p ./TOOLS/MHCI/
!tar -xzf $mhci_filename -C ./TOOLS/MHCI/
!rm $mhci_filename

for dir in os.listdir("./TOOLS/MHCI/"):
    if not dir.startswith("."):
        MHCI_dir = os.path.abspath(f"./TOOLS/MHCI/{dir}/")

print("Configuring MHC I tool...")
%cd -q $MHCI_dir
!./configure
%cd -q -

MHCI_script = f"{MHCI_dir}src/predict_binding.py"

Upload a text file with the alleles to be used, one allele per line.

E.g.:

SLA-1*0101

SLA-1*0201

SLA-1*0202

SLA-1*0401

SLA-1*0501

SLA-1*0601

In [ ]:
from google.colab import files
MHCI_alleles_upload = files.upload()
MHCI_alleles_filename = next(iter(MHCI_alleles_upload))

listMHCI_alleles = []
with open(MHCI_alleles_filename, "r") as f:
    for line in f:
        listMHCI_alleles.append(line.strip())

!mv $MHCI_alleles_filename $MHCI_dir

In [ ]:
def JoinCD8Peptides(cd8_dir):
    nonamers_files = [os.path.join(cd8_dir, x) for x in os.listdir(cd8_dir) if x.endswith("nonamers.fasta")]

    joinedPeptideList = []
    for fasta_file in nonamers_files:
        peptideList = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
        for x in peptideList:
            joinedPeptideList.append(x)
    uniquePeptideList = set(joinedPeptideList)

    CD8_peptides_file = os.path.join(cd8_dir, "ALL_PEPTIDES.fasta")
    with open(CD8_peptides_file, "w+") as f:
        for peptide in uniquePeptideList:
            f.write(f">{peptide}\n")
            f.write(peptide + "\n")
    
    return CD8_peptides_file

def MHCI(dir_CD8, allele_list):
    start_time = time.time()

    peptides_file = JoinCD8Peptides(dir_CD8)
    peptide_abspath = os.path.abspath(peptides_file)
    saveDir = createPath(os.path.abspath("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD8/MHCI_predictions/"))
    saveFile = os.path.join(saveDir, "mhci_output.txt")
    
    print("Calculating MHC I binding affinities")
    print("-----------------------------------")
    print("This might take a while...")
    print("-----------------------------------")

    %cd -q $MHCI_dir
    try:
        script_arguments = ["python",
                            "./src/predict_binding.py",
                            "IEDB_recommended",
                            ",".join(allele_list),
                            ",".join(['9' for x in allele_list]),
                            f"'{peptide_abspath}'",
                            ">",
                            f"'{saveFile}'",
                            ]

        mhci_script = " ".join(script_arguments)
        !$mhci_script

        %cd -q -

        elapsedtime = time.time() - start_time
        print("")
        print("Predictions for MHC I completed!")
        print("Total time: " + time.strftime("%H:%M:%S", time.gmtime(elapsedtime)))
        print("-------------------------------")
        print("")
    except:
        %cd -q -
        raise Exception("MHC I predictions failed.")
    

In [ ]:
MHCI("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8", listMHCI_alleles)

Finally we parse the output.

In [ ]:
def MHCIparser(mhci_results):
    with open(mhci_results, "r+") as r:
        for nline, line in enumerate(r):
            if line.startswith("allele"):
                skip_rows = nline
    df = pd.read_table(mhci_results, skiprows = skip_rows)
    df = df.drop_duplicates()

    MHCI_prediction_dict = {}
    for counter, peptide in enumerate(df["peptide"]):
        if peptide in MHCI_prediction_dict:
            MHCI_prediction_dict[peptide].append((df["allele"].iloc[counter], df["rank"].iloc[counter]))
        else:
            MHCI_prediction_dict[peptide] = [(df["allele"].iloc[counter], df["rank"].iloc[counter])]

    return MHCI_prediction_dict

In [ ]:
MHCI_predictions = MHCIparser('./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD8/MHCI_predictions/mhci_output.txt')

#### **Merge predictions**

In [ ]:
def epitopesByCluster(epitopes_dir):

    epitopes_files = [os.path.join(epitopes_dir, x) for x in os.listdir(epitopes_dir) if x.endswith("peptides9_nonamers.fasta")]

    dictionaryList = []
    for infile in epitopes_files:
        prefix = os.path.basename(infile).split("peptides9_nonamers.fasta")[0]
        values = {str(record.id):str(record.seq) for record in SeqIO.parse(infile, "fasta")}
        epidict = {prefix : values}
        dictionaryList.append(epidict)
    
    return dictionaryList

def fetchUniprotName(protein):
    uniprotID = protein.split("_")[2]
    url = 'https://www.uniprot.org/uniprot/' + uniprotID
    path_name = '//*[@id="content-protein"]/h1'

    try:
        response = requests.get(url)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path_name)

        if not tree:
            return [protein, "NA"]
        else:
            return [protein, str(tree[0].text_content())]
    except:
        return [protein, "NA"]

def Uniproter(protlist):    
    unique_prots = list(set(protlist))
    uniprotDict = {}
    results = ThreadPool(20).imap_unordered(fetchUniprotName, unique_prots)
    for r in results:
        uniprotDict[r[0]] = r[1]
    return uniprotDict

In [ ]:
def MHCI_results(epitopes_dir, MHCIpredictions_dict, TAPpredictions_dict):
    savePath = createPath("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD8/rawCSVs")
    epitopes_identities = epitopesByCluster("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8")

    for cluster_dict in epitopes_identities:
        for prefix, epi_data in cluster_dict.items():
            print(f"Merging values for {prefix}")
            print("----------------------------")
            dataframe_values = []
            print("Creating rows...", end = "")
            for id, seq in epi_data.items():
                for mhci_binding_data in MHCIpredictions_dict[seq]:
                    datarow = (seq,
                               id, 
                               TAPpredictions_dict[seq][0], #TAP%
                               TAPpredictions_dict[seq][1], #TAPIC50
                               mhci_binding_data[0], #allele
                               mhci_binding_data[1] #rank
                               )
                    dataframe_values.append(datarow)
            print("done!")

            print("Building dataframe...", end = "")
            df = pd.DataFrame(dataframe_values,
                              columns=["epitope",
                                       "protein",
                                       "TAP_prob",
                                       "TAP_IC50",
                                       "MHCI_allele",
                                       "MHCI_rank"],
                              dtype=float
                              )
            print("done!")
            
            print("Getting protein names...", end = "")
            protein_names = Uniproter([x for x in df["protein"]])
            df["protein name"] = df["protein"].map(protein_names)
            column_order = ["epitope",
                            "protein",
                            "protein name",
                            "TAP_prob",
                            "TAP_IC50",
                            "MHCI_allele",
                            "MHCI_rank"
                            ]
            df = df[column_order]
            print("done!")

            print("Saving dataframe to csv...", end = "")
            df.to_csv(os.path.join(savePath, prefix + "CD8.csv"))
            print("done!")
            print("")

In [ ]:
MHCI_results("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD8", MHCI_predictions, TAPdict)

#### **Formatting results**

In [ ]:
def FormatCD8(csv_file, MHCI_cutoff=1, TAP_cutoff=1):
    savePath = createPath("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD8")
    saveFile = os.path.join(savePath, os.path.basename(csv_file).replace(".csv", ".xlsx"))

    df = pd.read_csv(csv_file, index_col=0)

    #############################################################
    #Filter allele binding percentile. For MHC I, it's usually 1%
    df = df[df.MHCI_rank <= MHCI_cutoff]
    #############################################################
    
    ################################
    #Filter out all TAP IC50 above 1
    df = df[df.TAP_IC50 <= TAP_cutoff]
    ################################

    alleles_dict = {}
    TAP_dict = {}
    antigens_dict = {}
    epitopes_list = []

    for i in range(len(df)):
        epitope = df["epitope"].iloc[i]
        
        allele = df["MHCI_allele"].iloc[i]
        percentile = "(" + str(round(df["MHCI_rank"].iloc[i], 2)) + "%)"
        
        TAP_percent = df["TAP_prob"].iloc[i]
        TAP_IC50 = df["TAP_IC50"].iloc[i]
        
        antigen = "_".join(df["protein"].iloc[i].split("_")[:3])
        antigen_name = df["protein name"].iloc[i]
        
        #Add only unique epitopes to the list
        if not epitope in epitopes_list:
            epitopes_list.append(epitope)
        
        #Each epitope has specific allele percentiles
        if epitope in alleles_dict:
            alleles_dict[epitope].append(allele + " " + percentile)
        else:
            alleles_dict[epitope] = [allele + " " + percentile]
        
        TAP_dict[epitope] = (TAP_percent, TAP_IC50)

        #Each epitope comes from a different protein. Rarely, they can come
        #from more than one
        if epitope in antigens_dict:
            antigens_dict[epitope].append((antigen, antigen_name))
        else:
            antigens_dict[epitope] = [(antigen, antigen_name)]

    for key, values in alleles_dict.items():
        unique_alleles = list(set(values)) #Delete repeated alleles
        sorted_values = sorted(unique_alleles)
        alleles_dict[key] = sorted_values

    joined_alleles_dict = {}
    for key, values in alleles_dict.items():
        total_alleles = len(values)
        #We keep the allele, its percentile, and the total alleles binding
        joined_alleles_dict[key] = (" \n".join(values), total_alleles)

    unique_antigens_dict = {}    
    for key, values in antigens_dict.items():
        unique_antigens = list(dict.fromkeys(values)) #Delete repeated origins
        final_antigens = []
        for antigen in unique_antigens: #there might be more than one origin,
        #so we have to take into account that possibility
            final_antigens.append(antigen[0] + " (" + antigen[1] + ")")
        unique_antigens_dict[key] = "\n".join(final_antigens)

    tupledEpitopesList = [] #we will use a list of tuples because it's easier to
    #transform into a dataframe later
    for epitope in epitopes_list:   
        epitopeTuple = (epitope, 
                        unique_antigens_dict[epitope], #origin antigens
                        joined_alleles_dict[epitope][0], #alleles binding
                        joined_alleles_dict[epitope][1], #total alleles
                        round(TAP_dict[epitope][0], 2), #TAP %
                        round(TAP_dict[epitope][1], 2), #TAP IC50
                        )
        tupledEpitopesList.append(epitopeTuple)

    final_df = pd.DataFrame(tupledEpitopesList, columns=["epitope",
                                                        "protein",
                                                        "MHCI_alleles",
                                                        "MHCI_total",
                                                        "TAP_prob",
                                                        "TAP_IC50",
                                                        ]
                            )
    
    final_df.to_excel(saveFile, sheet_name = f"MHCI <={str(MHCI_cutoff)}% | TAP <={str(TAP_cutoff)}")

In [ ]:
epitopes_csv_dir = "./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD8/rawCSVs"
csv_files = [os.path.join(epitopes_csv_dir, x) for x in os.listdir(epitopes_csv_dir) if x.endswith(".csv")]

for csv in csv_files:
    FormatCD8(csv, MHCI_cutoff=1, TAP_cutoff=1)

### **CD4 - MHC II epitopes**

#### **Generate peptides of >= 15 amino acids**

In [ ]:
def Peptidator15(in_folder):

    shannon = in_folder.split("/")[-1].split("_")[-1]
    consensusFilesList = [os.path.join(in_folder, f) for f in os.listdir(in_folder) if f.endswith(".fasta")]

    for i in consensusFilesList:
        cluster = i.split("/")[-1].split("_")[0]
        consensusDict = {}
        with open(i,"r") as consensus:
            for line in consensus:
                if line.startswith(">"):
                    id = line.strip()
                    consensusDict[id] = ""
                else:
                    consensusDict[id] += line.strip()
                    
        for key, value in consensusDict.items():
            fragments = value.replace("_", " ").replace("-", " ").replace("*", " ").replace("X", " ").split()
            consensusDict[key] = [peptide for peptide in fragments if len(peptide) >= 15]


        destination = createPath("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD4/")       
        filename = cluster + "_" + shannon + "_peptides15.fasta"

        with open(destination + filename, "w+") as output:
            for key, values in sorted(consensusDict.items()):
                for counter, value in enumerate(values, 1):
                    output.write(key + "_" + str(counter))
                    output.write("\n")
                    output.write(value)
                    output.write("\n")

In [ ]:
for folder in os.listdir("./DATA/PROCESSED/T_PREDICTIONS/CONSENSUS"):
    if not folder.startswith("."):
        Peptidator15(os.path.join("./DATA/PROCESSED/T_PREDICTIONS/CONSENSUS", folder))

#### **Predicting peptide affinity for MHC II**

In [ ]:
mhcii_url = "https://downloads.iedb.org/tools/mhcii/3.1/IEDB_MHC_II-3.1.tar.gz"
mhcii_filename = mhcii_url.split("/")[-1]

print("Downloading IEDB MHC II tool...")
!wget $mhcii_url
!mkdir -p ./TOOLS/MHCII/
!tar -xzf $mhcii_filename -C ./TOOLS/MHCII/
!rm $mhcii_filename

for dir in os.listdir("./TOOLS/MHCII/"):
    if not dir.startswith("."):
        MHCII_dir = os.path.abspath(f"./TOOLS/MHCII/{dir}/")

print("Configuring MHC II tool...")
%cd -q $MHCII_dir
!./configure.py
%cd -q -

In [ ]:
MHC_II_alleles_url = "https://help.iedb.org/hc/en-us/article_attachments/360047583591/hla_ref_set.class_ii.txt"
MHC_II_alleles_file = os.path.basename(MHC_II_alleles_url)

!wget $MHC_II_alleles_url

listMHCII_alleles = []
with open(MHC_II_alleles_file, "r") as f:
    for line in f:
        listMHCII_alleles.append(line.strip())

!mv $MHC_II_alleles_file $MHCII_dir

In [ ]:
def JoinCD4Peptides(cd4_dir):
    peptide_files = [os.path.join(cd4_dir, x) for x in os.listdir(cd4_dir) if x.endswith("peptides15.fasta")]

    joinedPeptideList = []
    for fasta_file in peptide_files:
        peptideList = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
        for x in peptideList:
            joinedPeptideList.append(x)
    uniquePeptideList = set(joinedPeptideList)

    CD4_peptides_file = os.path.join(cd4_dir, "ALL_PEPTIDES.fasta")
    with open(CD4_peptides_file, "w+") as f:
        for peptide in uniquePeptideList:
            f.write(f">{peptide}\n")
            f.write(peptide + "\n")
    
    return CD4_peptides_file

def MHCII(dir_CD4, allele_list):
    start_time = time.time()

    peptides_file = JoinCD4Peptides(dir_CD4)

    peptide_abspath = os.path.abspath(peptides_file)
    saveDir = createPath(os.path.abspath("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD4/MHCII_predictions/"))
    saveFile = os.path.join(saveDir, "mhcii_output.txt")
    
    print("Calculating MHC II binding affinities")
    print("-------------------------------------")
    print("This might take a while...")
    print("-------------------------------------")

    %cd -q $MHCII_dir

    script_arguments = ["python",
                        "mhc_II_binding.py",
                        "IEDB_recommended",
                        ",".join(allele_list),
                        f"'{peptide_abspath}'",
                        ">",
                        f"'{saveFile}'",
                        ]

    mhcii_script = " ".join(script_arguments)
    !$mhcii_script

    %cd -q -

    elapsedtime = time.time() - start_time
    print("")
    print("Predictions for MHC II completed!")
    print("Total time: " + time.strftime("%H:%M:%S", time.gmtime(elapsedtime)))
    print("-------------------------------")
    print("")

In [ ]:
MHCII("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD4", listMHCII_alleles)

In [ ]:
def MHCIIparser(mhcii_results, original_file):
    df = pd.read_table(mhcii_results)
    df = df.drop_duplicates()

    peptide_dict = {}
    with open(original_file, "r") as f:
        counter = 1
        for line in f:
            if line.startswith(">"):
                peptide_dict[counter] = line.strip()[1:]
                counter += 1
    
    df["original_peptide"] = df["seq_num"].map(peptide_dict)

    MHCII_prediction_dict = {}
    for counter, o_peptide in enumerate(df["original_peptide"]):
        if o_peptide in MHCII_prediction_dict:
            MHCII_prediction_dict[o_peptide].append((df["peptide"].iloc[counter], df["allele"].iloc[counter], df["consensus_percentile_rank"].iloc[counter]))
        else:
            MHCII_prediction_dict[o_peptide] = [(df["peptide"].iloc[counter], df["allele"].iloc[counter], df["consensus_percentile_rank"].iloc[counter])]

    return MHCII_prediction_dict

In [ ]:
MHCII_predictions = MHCIIparser("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD4/MHCII_predictions/mhcii_output.txt",
                                "./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD4/ALL_PEPTIDES.fasta"
                                )

In [ ]:
def clustersMHC_II(fastas_dir):
    fileList = [os.path.join(fastas_dir, x) for x in os.listdir(fastas_dir) if x.endswith("peptides15.fasta")]

    for cluster_fasta in fileList:
        peptide_frags = [(record.id, record.seq) for record in SeqIO.parse(cluster_fasta, "fasta")]
        row_list = []
        for o_peptide in peptide_frags:
            origin = "_".join(o_peptide[0].split("_")[1:3])
            epitopes = MHCII_predictions[o_peptide[1]]
            for epitope in epitopes:
                row = (epitope[0], #epitope
                       origin, #protein
                       epitope[1], #allele
                       epitope[2], #rank
                       )
                row_list.append(row)

        df = pd.DataFrame(row_list, columns=["epitope",
                                             "protein",
                                             "MHCII_allele",
                                             "MHCII_rank"]
                          )
        
        fileName = os.path.basename(cluster_fasta).replace(".fasta", ".csv")
        savePath = os.path.join("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD4", fileName)
        df.to_csv(savePath)

In [ ]:
clustersMHC_II("./DATA/PROCESSED/T_PREDICTIONS/PEPTIDES/CD4")

#### **Formatting results**

In [ ]:
def joinAlleles_MHC_II(df):
    
    alleles_dict = {}
    antigens_dict = {}
    epitopes_list = []

    for i in range(len(df)):
        epitope = df["epitope"].iloc[i]
        
        allele = df["MHCII_allele"].iloc[i]
        percentile = f"({str(round(df['MHCII_rank'].iloc[i], 2))}%)"
        antigen = df["protein"].iloc[i]
        
        #Add only unique epitopes to the list
        if not epitope in epitopes_list:
            epitopes_list.append(epitope)
        
        #Each epitope has specific allele percentiles
        if epitope in alleles_dict:
            alleles_dict[epitope].append(allele + " " + percentile)
        else:
            alleles_dict[epitope] = [allele + " " + percentile]

        #Each epitope comes from a different protein. Rarely, they can come
        #from more than one
        if epitope in antigens_dict:
            antigens_dict[epitope].append(antigen)
        else:
            antigens_dict[epitope] = [antigen]

    for key, values in alleles_dict.items():
        unique_alleles = list(set(values)) #Delete repeated alleles
        sorted_values = sorted(unique_alleles)
        alleles_dict[key] = sorted_values

    joined_alleles_dict = {}
    for key, values in alleles_dict.items():
        total_alleles = len(values)
        #We keep the allele, its percentile, and the total alleles binding
        joined_alleles_dict[key] = (" \n".join(values), total_alleles)

    unique_antigens_dict = {}    
    for key, values in antigens_dict.items():
        unique_antigens = list(set(values)) #Delete repeated origins

        if len(unique_antigens) == 1:
            unique_antigens_dict[key] = unique_antigens[0]
        else: #there might be more than one origin
            unique_antigens_dict[key] = "|".join(unique_antigens)

    tupledEpitopesList = [] #we will use a list of tuples because it's easier to
    #transform into a dataframe later
    for epitope in epitopes_list:   
        epitopeTuple = (epitope,
                    joined_alleles_dict[epitope][0], #alleles binding
                    joined_alleles_dict[epitope][1], #total alleles
                    unique_antigens_dict[epitope], #origin antigens
                    )
        tupledEpitopesList.append(epitopeTuple)

    final_df = pd.DataFrame(tupledEpitopesList, columns=["epitope",
                                                        "alleles",
                                                        "total alleles",
                                                        "antigen",
                                                        ]
                            )

    return final_df

def overlapEpitopes(df, proteins_file):

    antigen_dict = {}
    alleles_dict = {}
    antigen_names_dict = {}

    for i in range(len(df)):
        antigen = df["antigen"].iloc[i].split("|")
        epitope = df["epitope"].iloc[i]
        alleles = [x for x in df["alleles"].iloc[i].split() if not "%" in x]
        
        if len(antigen) == 1:
            if antigen[0] in antigen_dict:
                antigen_dict[antigen[0]].append(epitope)
            else:
                antigen_dict[antigen[0]] = [epitope]
        else:
            if antigen in antigen_dict:
                antigen_dict[antigen].append(epitope)
            else:
                antigen_dict[antigen] = [epitope]
        
        alleles_dict[epitope] = alleles

    #We need the original sequences of each protein to map the epitopes
    allProteins = {}
    with open(proteins_file, "r") as f:
        for line in f:
            if line.startswith(">"):
                proteinFullID=line.strip()[1:]
                allProteins[proteinFullID] = ""
            else:
                allProteins[proteinFullID] = allProteins[proteinFullID] + line.strip()

    #Map each epitope to the reference
    final_tupled_list = []
    for origin, epitopes in antigen_dict.items():
        
        # In the case of multiple origins
        if isinstance(origin, list):
            print(f"Epitopes {', '.join(epitopes)} could not be merged.")
            for epitope in epitopes:
                newTuple = (epitope, #epitope seq
                            origin, #protein of origin
                            ", \n".join(alleles_dict[epitope]), #alleles binding
                            str(len(alleles_dict[epitope])) #total alleles
                            )
                final_tupled_list.append(newTuple)

        # In the case of single origin
        else:
            refseq = allProteins[origin]

            #We map each epitope to the origin sequence to know the order
            epitope_order = {}
            for epitope in epitopes:
                start_aa = refseq.find(epitope)
                if start_aa == -1: #not found
                    raise Exception ("Epitope not found")
                epitope_order[start_aa] = epitope

            # We will go epitope by epitope and find if they overlap.
            # Any overlapping epitopes will be put into the same group
            set_of_epitopes = []
            current_set = {}
            current_pos = None
            current_endpos = None
            for start_aa, epitope in sorted(epitope_order.items()):  
                if current_pos == None: #First epitope
                    current_pos = start_aa
                    current_endpos = start_aa + len(epitope) - 1
                    current_set[start_aa] = epitope

                elif current_pos + 1 == start_aa: #Next consecutive epitope
                    current_pos = start_aa
                    current_endpos = start_aa + len(epitope) - 1
                    current_set[start_aa] = epitope
                
                elif current_pos + 1 < start_aa: #next epitope start is not consecutive...

                    if start_aa <= current_endpos: #...but it's still overlapping
                        current_pos = start_aa
                        current_endpos = start_aa + len(epitope) -1
                        current_set[start_aa] = epitope

                    else: #...and it is not overlapping with the previous one
                        set_of_epitopes.append(current_set) #close this group
                        current_set = {} #and empty for next group
                        current_pos = start_aa
                        current_endpos = start_aa + len(epitope) - 1
                        current_set[start_aa] = epitope
            set_of_epitopes.append(current_set)

            # Now we will create a fused epitope for each group.
            # We keep all the alleles that the individual epitopes could bind to
            fused_list = []
            current_fused = ""
            current_allele_list = []
            for epitope_group in set_of_epitopes:
                current_position = None
                for position, epitope in epitope_group.items():
                    if current_fused == "":
                        current_fused = epitope
                        current_position = position
                    else:
                        current_fused += epitope[current_position - position :]
                        current_position = position
                    
                    current_allele_list.extend(alleles_dict[epitope])
                unique_allele_list = list(set(current_allele_list))
                fused_list.append((current_fused, sorted(unique_allele_list)))
                current_fused = ""
                current_allele_list = []
            
            for fused_epitope in fused_list:
                newTuple = (fused_epitope[0], #epitope seq
                            origin, #protein of origin
                            ", \n".join(fused_epitope[1]), #alleles binding
                            len(fused_epitope[1]), #total alleles
                            )
                final_tupled_list.append(newTuple)
    
    final_df = pd.DataFrame(final_tupled_list, columns=["epitope", "protein", "MHCII_alleles", "MCHII_total_alleles"])

    return final_df

def fetchUniprotName(protein):
    uniprotID = protein.split("_")[1]
    url = 'https://www.uniprot.org/uniprot/' + uniprotID
    path_name = '//*[@id="content-protein"]/h1'

    try:
        response = requests.get(url)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path_name)

        if not tree:
            return [protein, "NA"]
        else:
            return [protein, str(tree[0].text_content())]
    except:
        return [protein, "NA"]

def Uniproter(protlist):    
    unique_prots = list(set(protlist))
    uniprotDict = {}
    results = ThreadPool(20).imap_unordered(fetchUniprotName, unique_prots)
    for r in results:
        uniprotDict[r[0]] = r[1]
    return uniprotDict

def processMHC_II(csv_file, MHC_II_cutoff, proteins_fasta="./DATA/RAW/allproteins.fa"):
    df = pd.read_csv(csv_file, index_col=0)
    df = df[df.MHCII_rank <= MHC_II_cutoff]

    df = joinAlleles_MHC_II(df)

    df = overlapEpitopes(df, proteins_fasta)

    df["protein name"] = df["protein"].map(Uniproter(df["protein"]))

    fileName = os.path.basename(csv_file).replace("_peptides15.csv", ".xlsx")
    savePath = "./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD4"
    saveFile = os.path.join(savePath, fileName)

    df.to_excel(saveFile, sheet_name=f"MHC II <={str(MHC_II_cutoff)}")

def formatMHC_II(csv_dir):
    csv_list = [os.path.join(csv_dir, x) for x in os.listdir(csv_dir) if x.endswith("peptides15.csv")]
    
    for csv_file in csv_list:
        processMHC_II(csv_file, 
                      MHC_II_cutoff = 0.1
                      )

In [ ]:
formatMHC_II("./DATA/PROCESSED/T_PREDICTIONS/EPITOPES/CD4")

---
# **3. Prediction of B-Cell Epitopes**

---

## **3a. Prediction of exposed proteins**

#### Importing list of exposed proteins

In [ ]:
from google.colab import files
%cd -q "./DATA/RAW"
uploaded = files.upload()
for k, v in uploaded.items():
    with open(k, "r") as f:
        exposedList=[]
        for line in f:
            exposedList.append(line.strip())
%cd -q -

In [ ]:
with open("./DATA/RAW/allproteins_EXP.fa","a+") as output:
    with open("./DATA/RAW/allproteins.fa", "r") as input:
        proteins = SeqIO.parse(input, "fasta")
        for protein in proteins:
            if protein.id in exposedList:
                output.write(f">E_{protein.id}\n")
                output.write(f"{protein.seq}\n")
            else:
                output.write(f">{protein.id}\n")
                output.write(f"{protein.seq}\n")

## **3b. Protein clustering**

### **CD-HIT**

We run the program *CD-HIT* to create protein clusters.

Options:

```
-i --> Input file
-o --> Output
-d 0 --> Full description of cluster name until first space
-c --> % identity cutoff threshold (set to 0.8, deafult is 0.9)
-s --> % of length for smaller sequences, we set to 0.75 to avoid strange alignments of short proteins (default 0)
-n --> word size
```





The input file must be a FASTA file with all the protein sequences to cluster.

For this pipeline, each entry has to be in the format `>PROTEOME-ID_PROTEIN-ID`.

e.g.:

```
>UP000000861_P0C9M9
MNSLQVLTKKVLIENKAFSNYHEDDSFILQQLGLWWENGPIGFCKQCKMVTGGSMLCSDVDSYELDRALVKAVKENQTDL
>UP000000861_P0C9Q0
MLPSLQSLTKKVLAGQCLPEDQHYLLKCYDLWWNNAPITFDHNLRLIKSAGLQEGLDLNMALVKAVKENNYSLIKLFTEW
>UP000000861_P0C9K9
MITLYEAAIKTLITHRKQILKHPDSREILLALGLYWNKTHILLKCHECGKISLTGKHSTKCININCLLILAIKKKNKRMV
>UP000000861_P0C8F5
MKMHIARDSIVYLLNKHLQNTILTNKIEQECFLQADTPKKYLQYIKPFLINCMTKNITTDLVMKDSKRLEPYITLEMRDI
```



In [ ]:
#We specify an identity cutoff at 80%, and a minimum length for smaller sequences of 75%

!cd-hit -i ./DATA/RAW/allproteins_EXP.fa -o "./DATA/PROCESSED/B_PREDICTIONS/globalclusters" -d 0 -c 0.8 -s 0.75

### **Cluster filtering**

We will filter our clusters according to:

1.   **Presence of an exposed protein**


2.   **Number of proteomes in each cluster**

    We want a minimum representation of all strains in our clusters.


3.   **Presence of a particular proteome in each cluster**

    In case we want a particular proteome to be present always.


In [ ]:
def parseClusters(clstrfile):
    returndict = {}
    print(f"Parsing clusters for file {os.path.basename(clstrfile)}:")
    with open(clstrfile, "r") as f:
        for line in f:
            if line.startswith(">"):
                clusterID = line.strip()
                returndict[clusterID] = []
            else:
                protein_ID = line.strip().split()[2]
                protein_ID = protein_ID[1:protein_ID.find("...")]
                returndict[clusterID].append(protein_ID)
    print(f"Total number of clusters: {str(len(returndict))}")
    return returndict

def filterExposedClusters(clstrfile,
                   min_proteomes, #Minimum number of proteomes in each cluster
                   mustProteome=""
                   ):
    
    clusterDict = parseClusters(clstrfile)
  
    for clusterID, proteins_list in list(clusterDict.items()): #make it a list so we can delete keys
        proteome_list = []
        exposedFlag = False
        for protein in proteins_list:

            if protein.startswith("E_"):
                exposedFlag = True

            proteome = protein[protein.find("UP"):protein.find("_", 2)]
            proteome_list.append(proteome)

        if not exposedFlag:
            del clusterDict[clusterID]

        unique_proteomes = list(set(proteome_list))
        if len(unique_proteomes) < min_proteomes:
            try:
                del clusterDict[clusterID]
            except:
                pass

        if mustHaveProteome:
            if not mustHaveProteome in unique_proteomes:
                try:
                    del clusterDict[clusterID]
                except:
                    pass

    print(f"Filtered number of clusters: {str(len(clusterDict))}")

    return clusterDict

In [ ]:
#In case we want a particular proteome to be present in all clusters, we can
#specify it here. Otherwise, leave it as an empty string.

mustHaveProteome = "UP000141072" #Georgia 2007/1

global_clusters = filterExposedClusters("./DATA/PROCESSED/B_PREDICTIONS/globalclusters.clstr",
                                 min_proteomes=14,
                                 mustProteome=mustHaveProteome
                                 )

### **Reconstructing FASTAS**

We will create a new FASTA file for each cluster, recreating the sequences.


In [ ]:
#In this case, we will give priority to proteomes from genotype II, the current
#responsible of ASFV epidemic. We will give two tiers of importance: first 
#Georgia, and then the rest of genotype II.

priorityProteomesList = ["UP000141072", #Georgia 2007/1
                    "UP000345145", #Georgia 2008/2
                    "UP000267045", #Estonia 2014
                    "UP000326051", #Lithuania 2014 
                    "UP000325567", #Moldova 2017
                    "UP000327056", #CzechRepublic 2017/1
                    "UP000290386", #China/2018/AnhuiXCGQ
                    "UP000291821", #China Pig/HLJ/2018
                    "UP000292678", #China DB/LN/2018
                    "UP000307568", #Belgium 2018/1
                    "UP000316600", #China ASFV-wbBS01
                    "UP000324915", #Belgium Etalle 2018
                    "UP000428265", #Hungary ASFV_HU_2018
                    "UP000422299", #China/CAS19-01/2019 
                    ]

priorityProteomesDict = {}
for proteome in priorityProteomesList:
    if proteome == "UP000141072":
        priorityProteomesDict[proteome] = "01"
    else:
        priorityProteomesDict[proteome] = "02"

In [ ]:
#We will parse the original file with all proteins in order to create a complete
#dictionary that contains all sequences in a format "proteinID : sequence"

with open("./DATA/RAW/allproteins.fa", "r") as f:
    allProteins = {record.id:str(record.seq) for record in SeqIO.parse(f, "fasta")}

In [ ]:
def reconstruct_fasta(clusterDictionary, savePath, proteomePriorityDict, allProteins=allProteins):
    clusterName = os.path.split(os.path.dirname(savePath))[-1]

    for clusterID, clusterProteins in clusterDictionary.items():
        
        fileName = f"{clusterName}_{clusterID[1:]}.fa"
        filePath = os.path.join(savePath, fileName) 

        with open(filePath, "w+") as f:
            for protein in clusterProteins:
                protein = protein[protein.find("UP"):]
                proteome, proteinID = protein.split("_")

                #Here we asign each proteome a priority by adding a prefix
                if proteome in proteomePriorityDict:
                    f.write(f">{proteomePriorityDict[proteome]}_{proteome}_{proteinID}\n")
                else:
                    f.write(f">99_{proteome}_{proteinID}\n")
                f.write(allProteins[protein]+"\n")

    print (f"FASTAS for {clusterName} reconstructed.")

In [ ]:
globalPath = createPath("./DATA/PROCESSED/B_PREDICTIONS/FASTAS CLUSTERS/GLOBAL/")
reconstruct_fasta(global_clusters, globalPath, priorityProteomesDict)

## **3c. Multiple sequence alignment**

### **Running MUSCLE**

We run the MUSCLE MSA tool for each cluster. It results in a new MSA fasta file stored in MUSCLE ALIGNMENTS.

In [ ]:
def runMUSCLE(infile, outfile):
    #Here is where we installed MUSCLE
    muscle_exe = r"/usr/local/bin/muscle3.8.31_i86linux64"

    muscle_cline = MuscleCommandline(muscle_exe,
                                     input=infile,
                                     out=outfile,
                                     clwstrict=True
                                    )
    muscle_cline()

def alignDir(fastas_dir):
    dirname = os.path.basename(os.path.normpath(fastas_dir))

    print ("Aligning " + dirname + " clusters")
    print ("-----------------------------------")

    out_path = createPath("./DATA/PROCESSED/B_PREDICTIONS/MUSCLE ALIGNMENTS/UNSORTED/" + dirname)

    for root, dirs, files in os.walk(fastas_dir):
        for fasta in tqdm(files):
            if fasta.endswith(".fa"):
                in_file = os.path.join(root, fasta)
                out_file = out_path + "/MSA_" + fasta
                runMUSCLE(in_file, out_file)
    print("-------------------------------------")    
    print ("All MSAs for "+ str(dirname) +" done!")
    print("-------------------------------------")
    print()
    print()

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/B_PREDICTIONS/FASTAS CLUSTERS/"):
    for dir in dirs:
        if not dir.startswith("."):
            alignDir(os.path.join(root, dir))

### **Sorting the MSAs**

We will now open each MSA and sort it first according to our proteome priority, and then by number of positions aligned.

In [ ]:
def keyMSA(item):
    #We need a custom function to sort the MSA by priority and length of alignment
    returnList = []
    returnList.append(1 - int(item.id.split("_")[0])) #(1 - x) because descending order
    returnList.append(len(item) - item.seq.count("-"))
    return tuple(returnList)

def sortMSA(msafile):
    alignment = AlignIO.read(msafile,"clustal")
    alignment.sort(key = keyMSA, reverse=True)
    return alignment

def sortMSA_dir(in_folder):
    musclefiles = [x for x in os.listdir(in_folder) if x.endswith(".fa")]
    clusterName = os.path.basename(os.path.normpath(in_folder))
    dir_sorted = createPath(f"./DATA/PROCESSED/B_PREDICTIONS/MUSCLE ALIGNMENTS/SORTED/{clusterName}/")

    for m in musclefiles:
        infile = os.path.join(in_folder,m)
        aln = sortMSA(infile)
        with open(os.path.join(dir_sorted, m[:-3] + "_sorted.aln"),"w+") as outfile:
            AlignIO.write(aln, outfile, "clustal")

    print(f"Sorted {str(len(musclefiles))} {clusterName} alignments")

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/B_PREDICTIONS/MUSCLE ALIGNMENTS/UNSORTED"):
    for dir in dirs:
        if not dir.startswith("."):
            msa_dir = os.path.join(root,dir)
            sortMSA_dir(msa_dir)

## **3d. Entropy and consensus sequence**

### **Calculate entropy and generate consensus sequences**

We will use the Shannon entropy to calculate the variability in each position of each MSA. If a position has an entropy above the selected threshold (by default 0), it will be masked with an asterisk(*), thus generating a masked consensus sequence.

Then, we will find the sequence from the MSA that most closely resembles the masked consensus, and mask the same positions as the masked consensus. This way, we end up with a "real" masked sequence, not a consensus "artificial" one.

In [ ]:
def shannonEntropy(data):
    data_series = pd.Series(data)
    counts = data_series.value_counts()
    shannon_entropy = scipy.stats.entropy(counts)
    return shannon_entropy

def Shannon(alnfile, shannon_threshold=0):   
    clusterName = os.path.dirname(alnfile).split("/")[-1]
    shannon_str = str(shannon_threshold).replace(".", "")
    dest_pref = f"./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/SHANNON_{shannon_str}"
    dest_dir = createPath(f"{dest_pref}/{clusterName}/")
    dest_file = os.path.basename(alnfile).replace("sorted.aln",
                                                  f"consensus{shannon_str}.fasta"
                                                )
    
    #It will go position by position, calculate the entropy and then create a
    #consensus sequence with the most common amino acid, unless the entropy is 
    #higher than the threshold, in which case it will insert an *.
    consensus_list = []
    alignment = AlignIO.read(alnfile, "clustal")
    for i in range(0,alignment.get_alignment_length()):
        aa = []
        for record in alignment:
            aa.append(record.seq[i])
        position_entropy = shannonEntropy(aa)
        if position_entropy <= shannon_threshold:
            consensus_list.append(Counter(aa).most_common()[0][0])
        else:
            consensus_list.append("*")
    consensus_seq = "".join(consensus_list)

    #To find the closest sequence to the consensus, we will do a pairwise alignment
    #between the consensus and all sequences from the MSA. The sequence with highest
    #score will be the closest one. In case of equal score, prioritized proteomes
    #have advantage.
    refSeq = None
    highestScore = 0
    for record in alignment:
        pairwise_score = pairwise2.align.globalxx(record.seq,
                                                consensus_seq,
                                                score_only=True
                                                )
        if pairwise_score > highestScore:
            highestScore = pairwise_score
            refSeq = record
        elif pairwise_score == highestScore:
            if int(record.id[0:2]) < int(refSeq.id[0:2]):
                refSeq = record
    
    #Finally, we will mask the positions of the closest sequence according to
    #the masked consensus sequence.
    refSeq_aa = list(refSeq.seq)
    masked_ref = ["*"] * len(refSeq_aa)
    for i in range(0, len(refSeq_aa)):
        if consensus_seq[i] != "*":
            masked_ref[i] = refSeq_aa[i]
    
    #We write the results in a new fasta file
    with open(dest_dir + dest_file, "w+") as f:
        f.write(f">{refSeq.id}\n")
        f.write("".join(masked_ref))

    #All results from the same cluster type and Shannon threshold will be appended
    #in a file.
    mergedFile = (f"{dest_pref}/{clusterName}_fullconsensus{shannon_str}.fasta")
    with open(mergedFile, "a+") as f:
        f.write(f">{refSeq.id}\n")
        f.write("".join(masked_ref)+"\n")

In [ ]:
#In case we want to calculate the consensus for different thresholds, we can
#put them in a list here:
shannon_thresholds = [0,
                      #0.25,
                      #0.5
                      ]

for s in shannon_thresholds:
    for dir in os.listdir("./DATA/PROCESSED/B_PREDICTIONS/MUSCLE ALIGNMENTS/SORTED"):
        if not dir.startswith("."):
            cluster_dir = os.path.join("./DATA/PROCESSED/B_PREDICTIONS/MUSCLE ALIGNMENTS/SORTED", dir)
            print(f"Calculating invariable proteome at Shannon {s} for {dir}")
            for f in tqdm(os.listdir(cluster_dir)):
                if f.endswith(".aln"):
                    Shannon(os.path.join(cluster_dir, f), s)

## **3e. Validation of IEDB B-cell epitopes**

### **Downloading IEDB epitopes**

We will download the available confirmed epitopes of our organism from IEDB via Selenium.

In [ ]:
#We need the ID of our organism at IEDB. For example, for ASFV is 10497, and for
#T. cruzi is 5693. It can be found at IEDB.org and searching in "Organism".

IEDB_ID = 10497

In [ ]:
%cd -q "./DATA/RAW"

wd = webdriver.Chrome('chromedriver', options=options)
wd.get(f"http://www.iedb.org/sourceOrgId/{IEDB_ID}")

wd.find_element_by_xpath('//*[@id="content"]/div/div[2]/table/tbody/tr[4]/td[2]/div/a').click() #Click on B assays link
time.sleep(5)

wd.find_element_by_css_selector('#refine_container > div:nth-child(8) > div.search_content > div.ui_input.check.search_bothlinks > div.checkbox.square').click() #Positive assays only
time.sleep(2)

wd.find_element_by_css_selector('#refine_container > button:nth-child(4)').click() #Search
time.sleep(2)

wd.find_element_by_css_selector('#result_carousel > div.carousel_content.active > div > div.content_tabholder > div:nth-child(2) > div.content_tab > div.content_text').click() #B cell tab
time.sleep(2)

wd.find_element_by_css_selector('#result_carousel > div.carousel_content.active > div > div.contenttable_content.active > div > div:nth-child(1) > div.exportholder').click() #Export results
time.sleep(2)

wd.find_element_by_css_selector("div.inwindow_popup.drop_arrow.identifier_container.show > div > div:nth-child(1) > div.exportholder > div.txt").click() #Popup Export to CSV file
time.sleep(2)

wd.switch_to.window(wd.window_handles[-1]) #Switch to "download" page
time.sleep(2)

wd.refresh() #We refresh to prompt the download
time.sleep(2)

wd.close()

%cd -q ../../

In [ ]:
for f in os.listdir("./DATA/RAW/"):
    if f.endswith(".zip"):
        if f.startswith("bcell"):
            zipname = "./DATA/RAW/" + f
            !unzip $zipname -d "./DATA/RAW/"
            os.remove(zipname)

for f in os.listdir("./DATA/RAW/"):
    if f.endswith(".csv"):
        if f.startswith("bcell"):
            os.rename(r"./DATA/RAW/" + f, r"./DATA/RAW/B_Cell_IEDB_epitopes.csv")

In [ ]:
IEDB_B_epitopes = {}

with open("./DATA/RAW/B_Cell_IEDB_epitopes.csv","r") as f:
    reader = csv.reader(f)
    next(reader)
    next(reader)
    for line in reader:
        IEDB_B_epitopes[line[0]] = (line[11], #Sequence
                                    line[15], #Antigen
                                    line[17], #Antigen ID
                                    line[71], #Assay 1
                                    line[72], #Assay 2
                                    line[74]  #Assay 3
                                    )

### **Identify conserved IEDB epitopes**

In [ ]:
def IEDB_conserved(consensus_file, epitopes_dict):
    conserved_records = list(SeqIO.parse(consensus_file, "fasta"))
    conservedEpis = []

    for record in conserved_records:
        sequence = str(record.seq)
        for epitope_id, epitope_data in epitopes_dict.items():
            if epitope_data[0] in sequence:
                epitope_entry = [epitope_id,
                                 epitope_data[0],
                                 record.id,
                                 epitope_data[1],
                                 epitope_data[2],
                                 " ".join(epitope_data[3:])]
                conservedEpis.append(tuple(epitope_entry))

    df = pd.DataFrame(conservedEpis,
                      columns=['Epitope_ID',
                               'Epitope',
                               'Antigen',
                               'IEDB_Antigen name',
                               'IEDB_Antigen ID',
                               'IEDB_Assay'
                               ]
                      )

    clusterN = os.path.basename(consensus_file).split("_")[0]
    shannonN = os.path.dirname(consensus_file).split("/")[-1].split("_")[-1]

    output_path = createPath(consensus_file.split("CONSENSUS")[0] + "IEDB/")
    output_path = f"{output_path}{clusterN}_{shannonN}_IEDB_epis.csv"

    df.to_csv(output_path)

In [ ]:
consensusFolder = "./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/"

for shannon in os.listdir(consensusFolder):
    for f in os.listdir(consensusFolder + shannon):
        if f.endswith(".fasta"):
            IEDB_conserved(os.path.join(consensusFolder + shannon, f), IEDB_B_epitopes)

## **3f. *De novo* structure-based epitope predictions**

### **Finding and downloading PDB structures**

#### PDB BLAST

In [ ]:
with open(createPath("./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/") + "ExposedSequences.fasta", "w+") as output:
    with open("./DATA/RAW/allproteins.fa", "r") as input:
        allProteins = {record.id:str(record.seq) for record in SeqIO.parse(input, "fasta")}
        for protein in exposedList:
            output.write(f">{protein}\n")
            output.write(allProteins[protein] + "\n")

In [ ]:
exposedFasta = open('./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/ExposedSequences.fasta')

result_handle = NCBIWWW.qblast("blastp",
                               "pdb",
                               exposedFasta.read(),
                               )

with open('./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/PDB_BLAST_results.xml', 'w+') as save_file: 
    blast_results = result_handle.read() 
    save_file.write(blast_results)

exposedFasta.close()

In [ ]:
def PDBblast_parser(xml, min_identity=0.9):
    handle = open(xml)
    records = NCBIXML.parse(handle)
    
    best_hits = {}
    
    for record in records:
        epitope_length = record.query_length
        
        hits = []
        
        if record.alignments:
            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    if float(hsp.identities) / int(epitope_length) >= min_identity: #we want a minimum identity
                        hits.append((alignment.title.split("|")[1],
                                    float(hsp.identities) / int(epitope_length)
                                    *100,
                                    hsp.expect,
                                    ))
            if hits:                    
                best_hits[record.query] = [x for x in hits]
            
    handle.close()
                
    return best_hits

#### Download .pdb files

In [ ]:
for key, values in PDBblast_parser("./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/PDB_BLAST_results.xml").items():
    for value in values:
        PDB_id = value[0]
        url = f"https://files.rcsb.org/download/{PDB_id}.pdb"
        !wget $url

listPDBfiles = [x for x in os.listdir() if x.endswith(".pdb")]
pdb_dir = createPath("./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/PDBs")
for pdb in listPDBfiles:
    !mv $pdb '$pdb_dir'

### **PDB structures analysis**

#### Installing NACCESS

In [ ]:
#Upload the already decrypted tar.gz file
!mkdir -p ./TOOLS/NACCESS/
from google.colab import files
naccess_pack = files.upload()
naccess_filename = next(iter(naccess_pack))
!tar -xzf {naccess_filename} -C ./TOOLS/NACCESS
!rm {naccess_filename}

for dir in os.listdir("./TOOLS/NACCESS/"):
    if not dir.startswith("."):
        naccessDir = f"./TOOLS/NACCESS/{dir}"

#Modify the NACCESS script to patch a common error
accall_lines = []
with open(os.path.join(naccessDir, "accall.f"), "r") as f:
    for line in f:
        accall_lines.append(line.strip("\n"))
correctedLine = "                  write(4,*)"
accall_lines[254] = correctedLine

with open(os.path.join(naccessDir, "accall.f"), "w+") as f:
    for line in accall_lines:
        f.write(line + "\n")

#Finally, install the program
%cd -q $naccessDir
!chmod +x install.scr
!chmod +x naccess.scr
!csh install.scr
%cd -q -

#### Process PDB files

In [ ]:
def B_structure_based_epitopes(pdbfile, consensusFile):

    PDB_id = os.path.basename(pdbfile).split(".")[0]

    #Run NACCESS
    !./TOOLS/NACCESS/naccess2.1.1/naccess $pdbfile

    #Parse the .rsa file generated by NACCESS
    RSA_file = f"{PDB_id}.rsa"
    RSA_values = {}
    with open(RSA_file, "r") as input:
        for line in input:
            if line.startswith("RES"):
                aa = line[4:7].strip()
                chain = line[8:9].strip()
                position = int(line[9:13].strip())
                rsa = float(line[23:29].strip())
                RSA_values[(chain, position)] = (aa, rsa)
    
    #Move all naccess-generated files
    destNaccess = createPath("./DATA/PROCESSED/B_PREDICTIONS/STRUCTURE_BASED/NACCESS_results/")
    naccess_files = [x for x in os.listdir() if x.startswith(PDB_id)]
    for f in naccess_files:
        !mv $f $destNaccess
    
    #Parse the B values from the PDB file
    PDB_values = {}
    with open(pdbfile, "r") as input_f:
        for line in input_f:
            if line.startswith("ATOM"):
                data = line.strip()
                if "CA" in data:
                    try:
                        PDB_values[(data[21:22].strip(), int(data[23:30].strip()))] = (data[17:20], float(data[60:66].strip()))
                    except:
                        raise Exception("Something went wrong with the PDB file")

    #Normalize the B factors. For this we need to calculate first
    #the mean of all B values as well as their standard deviation.
    B_values = []
    for key, value in PDB_values.items():
        B_values.append(value[1])     
    a = np.array(B_values)
    B_mean = a.mean()
    B_std = a.std()
    
    #Change all the PDB raw values for their normalized ones
    PDB_normalized = {}
    for key, value in PDB_values.items():
        normB = (value[1] - B_mean) / B_std
        PDB_normalized[key] = (value[0], normB)

    #Search for flexible regions according to B-factor calculations
    flexible_regions = []
    current_region = []

    for key, value in sorted(PDB_normalized.items()):
        if value[1] >= 1: #We want a minimum of 1 normalized flexibility
            current_region.append((key[0], key[1], value[0], value[1]))
        else:
            if len(current_region) >= 8: #minimum epitope length
                flexible_regions.append(current_region)
                current_region = []
            else:
                current_region = []

    #Add RSA values to these regions
    potential_regions = []

    for region in flexible_regions:
        potential_region = []

        for position in region:
            if RSA_values[(position[0], position[1])][0] == position[2]:
                valuesTuple = (position[0], position[1], position[2], position[3], RSA_values[(position[0], position[1])][1])
                potential_region.append(valuesTuple)
            else:
                raise Exception("Aminoacids do not match")  

        potential_regions.append(potential_region)

    #We will change the aminoacid names for their one-letter code
    d_aa = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K', 
        'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
        'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
        'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    #For each region, we will select all fragments with the required rsa and flex
    fragments_list = []

    for region in potential_regions:
        epis = []
        
        for position_n, position in enumerate(region):
            for i in range(len(region) + 1):
                if position_n + i <= len(region):
                    
                    current_chain = position[0] + " " + str(position[1])
                    current_epi = region[position_n:i]  
                    epi_seq = "".join([d_aa[x[2]] for x in current_epi])
                    
                    if len(epi_seq) >= 8:   
                        
                        mean_rsa = np.array([x[4] for x in current_epi]).mean()
                        mean_flex = np.array([x[3] for x in current_epi]).mean()

                        if mean_rsa >= 50: #minimum RSA
                            if mean_flex >= 1: #minimum flex
                                epis.append((epi_seq, current_chain, mean_flex, mean_rsa))
            
        fragments_list.append(epis)

    #We have to see if these epitopes are conserved in the shannon analysis
    hits_dict = defaultdict(list)
    
    invariable_sequences = [(str(x.seq), str(x.id)) for x in list(SeqIO.parse(consensusFile, "fasta"))]

    conserved_list = []
    for fragments in fragments_list:
        conserved_fragments = []
        for fragment in fragments:
            for sequence in invariable_sequences:
                if fragment[0] in sequence[0]:
                    hits_dict[fragment[0]].append(sequence[1])
                    conserved_fragments.append(fragment)
        conserved_list.append(conserved_fragments)


    #We keep the largest epitope of each fragment. Ideally, at this point we should
    #check all fragments with BLAST first. However, it's more likely that larger
    #fragments have a lower identity with any given protein.

    B_epis = []
    for epi_list in conserved_list:
        if epi_list:
            sorted_epis = sorted(epi_list, key=lambda x: len(x[0]), reverse=True)
            B_epis.append(sorted_epis[0])

    #We create a Dataframe with the final epitopes, and add the consensus hits

    df = pd.DataFrame(B_epis, columns=["epitope", "chain", "mean flex", "mean rsa"])
    df['origin'] = df["epitope"].map(hits_dict)

    return df

In [ ]:
pdbfiles = [os.path.join(pdb_dir, x) for x in os.listdir(pdb_dir) if x.endswith(".pdb")]
consensusFolders =[f"./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/{x}/" for x in os.listdir("./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS") if x.startswith("SHANNON")]
for consensus_folder in consensusFolders:
    consensusFiles = [os.path.join(consensus_folder, x) for x in os.listdir(consensus_folder) if x.endswith(".fasta")]
    for consensus_file in consensusFiles:
        shannon = os.path.normpath(consensus_folder).split("/")[-1] 
        group = os.path.basename(consensus_file).split("_")[0]

        destPath = createPath("./DATA/PROCESSED/B_PREDICTIONS/EPITOPES/STRUCTURE_BASED/")
        destFile = destPath + group + "_" + shannon
        df = pd.DataFrame()

        for pdb_f in pdbfiles:
            df_pdb = B_structure_based_epitopes(os.path.abspath(pdb_f), consensus_file)
            df_pdb["PDB_ID"] = os.path.basename(pdb_f).split(".")[0]
            df = df.append(df_pdb, ignore_index = True)

        df.to_excel(destFile + ".xlsx")

CONTINUE HERE

## **3g. *De novo* sequence-based epitope predictions**

### **Install BepiPred2**

#### Installing prerequisites

BLAST

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/legacy.NOTSUPPORTED/2.2.18/blast-2.2.18-x64-linux.tar.gz
!mkdir -p ./TOOLS/BLAST/
!tar -xzf blast-2.2.18-x64-linux.tar.gz -C ./TOOLS/BLAST/
!rm blast-2.2.18-x64-linux.tar.gz
blastpgp_path = os.path.abspath("./TOOLS/BLAST/blast-2.2.18/bin/blastpgp")
blast_data_path = os.path.abspath("./TOOLS/BLAST/blast-2.2.18/data")

NetSurfP

In [ ]:
#Upload the .tar.gz file, uncompress and untar it.
!mkdir -p ./TOOLS/NETSURFP/
from google.colab import files
netsurfp_pack = files.upload()
netsurfp_filename = next(iter(netsurfp_pack))
!tar -xzf {netsurfp_filename} -C ./TOOLS/NETSURFP
!rm {netsurfp_filename}

for dir in os.listdir("./TOOLS/NETSURFP/"):
    if not dir.startswith("."):
        netsurfpDir = os.path.abspath(f"./TOOLS/NETSURFP/{dir}")
        netsurfpScript = f"{netsurfpDir}/netsurfp"
        netsurfpTmp = f"{netsurfpDir}/tmp"

!wget http://www.cbs.dtu.dk/services/NetSurfP-1.0/data.tar.gz
!tar -xzf data.tar.gz -C $netsurfpDir
!rm data.tar.gz
!wget http://www.cbs.dtu.dk/services/NetSurfP-1.0/nr70_db.tar.gz
!tar -xzf nr70_db.tar.gz -C $netsurfpDir
!rm nr70_db.tar.gz

In [ ]:
#Modify the NetSurfP script settings following the README file instructions
parsedNetSurf = []
with open(netsurfpScript, "r") as f:
    for line in f:
        parsedNetSurf.append(line.strip("\n"))

for counter, line in enumerate(parsedNetSurf):
    if line.startswith("  $ENV{NetSurfP}"):
        parsedNetSurf[counter] = "  $ENV{NetSurfP}	= '%s';" % (netsurfpDir)
    if line.startswith("  $ENV{BLASTPROG}"):
        parsedNetSurf[counter] = "  $ENV{BLASTPROG}	= '%s';" % (blastpgp_path)
    if line.startswith("  $ENV{BLASTMAT}"):
        parsedNetSurf[counter] = "  $ENV{BLASTMAT}	= '%s';" % (blast_data_path)
    if line.startswith("  $ENV{TMPDIR}"):
        parsedNetSurf[counter] = "  $ENV{TMPDIR}		= '%s';" % (netsurfpTmp)
    
with open(netsurfpScript, "w+") as f:
    for line in parsedNetSurf:
        f.write(line + "\n")

!chmod 755 {netsurfpScript}

Also outdated packages

In [ ]:
!pip install scipy==1.2.3 --no-cache-dir
!pip install numpy==1.16.6 --no-cache-dir
!pip install  matplotlib==2.0.0 --no-cache-dir
!pip install scikit-learn==0.17 --no-cache-dir

#### Downloading and installing the IEDB B-cell tool

In [ ]:
b_url = "https://downloads.iedb.org/tools/bcell/3.1/IEDB_BCell-3.1.tar.gz"
b_filename = b_url.split("/")[-1]

print("Downloading IEDB B-cell tool...")
!wget $b_url
!mkdir -p ./TOOLS/BCELL/
!tar -xzf $b_filename -C ./TOOLS/BCELL/
!rm $b_filename

for dir in os.listdir("./TOOLS/BCELL/"):
    if not dir.startswith("."):
        bcell_dir = os.path.abspath(f"./TOOLS/BCELL/{dir}")
        bcell_script = f"{bcell_dir}/predict_antibody_epitope.py"

%cd -q $bcell_dir

%env NETSURFP_BIN=$netsurfpScript
!./configure

%cd -q -

### **Run BepiPred2**

First, we create a new unmasked file with the consensus sequences that we will feed to the prediction algorithm.

In [ ]:
def getConsensus(in_dir):
    shannon = in_dir.split("_")[-1]   
    for f in os.listdir(in_dir):
        if f.endswith(".fasta"):
            output = f.replace(".fasta", "_unmasked.fasta")
            prots = [str(record.id) for record in SeqIO.parse(os.path.join(in_dir, f), "fasta")]
            with open(os.path.join(in_dir, output), "w+") as output_f:
                for protein in prots:
                    output_f.write(">" + protein + "\n")
                    output_f.write(allProteins[protein[3:]] + "\n")

In [ ]:
for subdir in os.listdir("./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS"):
    getConsensus(os.path.join("./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS", subdir))

Then, we define a function that will take our unmasked file and pass it through Bepipred.

In [ ]:
def Bepipred_prediction(filepath):
    starttime = time.time()
    rootdir = os.path.dirname(filepath)
    basename = os.path.basename(filepath)
    savedir = createPath("./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/")

    shannon = rootdir.split("/")[-1].split("_")[-1]
    clusterName = basename.split("_")[0]

    print("Predicting antibody binding for " + clusterName + " " + shannon)
    print("---------------------------------" + len(clusterName)*"-" + len(shannon)*"-")
    print("---------------------------------" + len(clusterName)*"-" + len(shannon)*"-")

    BepipredArgs=['python',
                  bcell_script,
                  '-m Bepipred2',
                  '-f',
                  f'"{filepath}"',
                  '>',
                  f'"{savedir}{clusterName}_{shannon}_bpresults.txt"'
                  ]

    script=" ".join(BepipredArgs)
    !$script

    elapsedtime = time.time() - starttime
    print("")
    print("Predictions for " + clusterName + " " + shannon + " completed!")
    print("Total time: " + time.strftime("%H:%M:%S", time.gmtime(elapsedtime)))
    print("-----------------" + len(clusterName)*"-" + len(shannon)*"-" + "-----------")
    print("")

Finally, we run the predictions.

In [ ]:
for root, dirs, files in os.walk("./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/"):
    for f in files:
        if f.endswith("unmasked.fasta"):
            Bepipred_prediction(os.path.abspath(os.path.join(root,f)))

#### **Parsing the output**

In [ ]:
def getUnmaskedAlignment(input_file, output_file):
    muscle_exe = r"/usr/local/bin/muscle3.8.31_i86linux64"
    muscle_cline = MuscleCommandline(muscle_exe, input = input_file, out = output_file, clwstrict=True)
    muscle_cline()
    
    alignment = AlignIO.read(output_file, "clustal")
    
    for record in alignment:
        if record.id.endswith("unmasked"):
            alignedSeq = str(record.seq)
        else:
            baseSeq = str(record.seq)

    return alignedSeq

def BepiParse(results_file, bp_threshold=0.5):
    basename = os.path.basename(results_file)
    clusterName = basename.split("_")[0]
    shannon = basename.split("_")[1]
    
    output_parsed = f"./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/{clusterName}_{shannon}_bepiparsed.fasta"
    open(output_parsed,"w+").close()
    
    bepipred_dict = {}
    with open(results_file, "r") as f:
        for line in f:
            if line.startswith("input"):
                seqFlag = False
                currentID = line.strip()[7:]
                bepipred_dict[currentID] = []
            elif line.startswith("Position"):
                seqFlag = True
            else:
                if seqFlag:
                    lineTuple = tuple(line.split())
                    bepipred_dict[currentID].append(lineTuple)
    
    #We need the masked and unmasked sequences of each protein
    masked_file = f"./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/SHANNON_{shannon}/{clusterName}_fullconsensus{shannon}.fasta"   
    masked_dict = {record.id:str(record.seq) for record in SeqIO.parse(masked_file,"fasta")}

    unmasked_file = f"./DATA/PROCESSED/B_PREDICTIONS/CONSENSUS/SHANNON_{shannon}/{clusterName}_fullconsensus{shannon}_unmasked.fasta"
    unmasked_dict = {record.id:str(record.seq) for record in SeqIO.parse(unmasked_file,"fasta")}

    #Since the lengths of masked and unmasked proteins might be different, we need
    #to align them first and get the unmasked alignment.
    for protein, values in bepipred_dict.items():
        muscle_input = "./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/muscletemp_i.fasta"
        muscle_output = "./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/muscletemp_o.fasta"
        
        with open(muscle_input, "w+") as f:
            f.write(">_masked\n")
            f.write(masked_dict[protein].replace("*", "X").replace("-", "X") + "\n")
            f.write(">_unmasked\n")
            f.write(unmasked_dict[protein] + "\n")


        unmaskedSeq = getUnmaskedAlignment(muscle_input, muscle_output)
        
        parsedSeq = ""
        
        #We have two counters, one for the Bepipred results and the other for
        #the protein sequence. 
        bpred_position = 0
        for position, aa in enumerate(unmaskedSeq):
            #In case the position is a gap. Bepipred counter does not move, since
            #there are no gaps there.
            if aa == "-":
                parsedSeq += aa
                continue

            #Check if the amino acids match, otherwise something wrong happened.
            elif not aa == values[bpred_position][1]:
                raise Exception("BepiPred results don't match unmasked sequence!")   

            #Amino acids match, but the position is masked.
            #Both counters move one position.  
            elif masked_dict[protein][position] in ["*", "-"]:
                parsedSeq += masked_dict[protein][position]
                bpred_position += 1
            
            #Amino acids match, now let's see what BepiPred predicts.
            #Both counters move one position. 
            elif float(values[bpred_position][2]) >= bp_threshold:
                parsedSeq += aa
                bpred_position += 1
            else:
                parsedSeq += "_"
                bpred_position += 1
        
        #Length of the parsed sequence should be the same as the unmasked seq.                        
        if not len(parsedSeq) == len(masked_dict[protein]):
            raise Exception("Parsed length does not match unmasked sequence")
              
        os.remove(muscle_input)
        os.remove(muscle_output)
            
        with open(output_parsed,"a+") as f:
            f.write(">" + protein + "\n")
            f.write(parsedSeq + "\n")

In [ ]:
for f in os.listdir("./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED"):
    if f.endswith("bpresults.txt"):
        BepiParse(f"./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/{f}")

### **Generate peptides of >=15 amino acids from prediction**

In [ ]:
def fetchUniprotName(protein):
    uniprotID = protein.split("_")[2]
    url = 'https://www.uniprot.org/uniprot/' + uniprotID
    path_name = '//*[@id="content-protein"]/h1'

    try:
        response = requests.get(url)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path_name)

        if not tree:
            return [protein, "NA"]
        else:
            return [protein, str(tree[0].text_content())]
    except:
        return [protein, "NA"]

def Uniproter(protlist):    
    unique_prots = list(set(protlist))
    uniprotDict = {}
    results = ThreadPool(20).imap_unordered(fetchUniprotName, unique_prots)
    for r in results:
        uniprotDict[r[0]] = r[1]
    return uniprotDict

def B_epitopes(bepiparsed_file):

    bepipredDict = {str(record.id) : str(record.seq) for record in SeqIO.parse(bepiparsed_file, "fasta")}

    epitopes_list = []
    for id, sequence in bepipredDict.items():
        fragments = sequence.replace("-", " ").replace("*", " ").replace("X", " ").replace("_", " ").split()
        for fragment in fragments:
            if len(fragment) >= 15:
                epitopes_list.append((fragment, id))
    
    destPath= createPath("./DATA/PROCESSED/B_PREDICTIONS/EPITOPES/SEQUENCE_BASED/")
    group = bepiparsed_file.split("/")[-1].split("_")[0]
    shannon = bepiparsed_file.split("/")[-1].split("_")[1]
    filename = f"{group}_{shannon}_Bseq.xlsx"

    df = pd.DataFrame(epitopes_list, columns = ["epitope", "protein"])

    uniprotNames = Uniproter(df["protein"])
    df["protein name"] = df["protein"].map(uniprotNames)

    df.to_csv(destPath + filename)

    return df

In [ ]:
bepiparsedList = [f"./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED/{f}" for f in os.listdir("./DATA/PROCESSED/B_PREDICTIONS/BEPIPRED") if f.endswith(".fasta")]
for f in bepiparsedList:
    B_epitopes(f)